# Archiving Social Media for Analysis (SQL Blog)




In this project, we archive social media data and integrate it with other data for analysis. Social Media platforms like Twitter, Instagram, Facebook, Youtube etc allow users an automated sentiment analysis, classification/auto-tagging, engagement analysis, reach analysis. The goal here is to create a database that stores syntactic and semantic information about tags. Inorder to do this, we create tables and keep adding information to it. Creating such a database involves designing a conceptual model first. In the second stage, we create tables, Add/Update the data. The third stage is where we write queries that answer the questions for a particular domain- movies- in this case using the social media platforms like Facebook, Instagram and Twitter. This portfolio consists:<br>

    o   Python scripts used to obtain the social media data.
    o   Data from a Twitter, Facebook, Instagram.
    o   SQL for all of your inserts and queries.
    
The domain choosen here is: Movies. A movies database and a cast database which contains feilds such as genres, production companies, movie name, character name, actor name, actor gender etc is used. Apart from this, we scrape social media websites like Twitter, Facebook and Instagram using their respective APIs to get more data and extract 'hashtags' with respect to each feild (person, place, thing-production company/genres). This data is put into separate tables as shown in the schema below and then linked with the movies and cast databse. Additionally, we answer the following questions in order to analyze this databse using SQL queries:<br>

    i. What are people saying about me (somebody)?
    ii. How viral are my posts?
    iii. How much influence to my posts have?
    iv. What posts are like mine?
    v. What users post like me?
    vi. Who should I be following?
    vii. What topics are trending in my domain?
    viii. What keywords/ hashtags should I add to my post?
    ix. Should I follow somebody back?
    x. What is the best time to post?
    xi. Should I add and picture or url to my post?
    xiii. What’s my reach?




In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import json


In [145]:
from bson.json_util import dumps

## Reading the main 'Movies' databse

In [2]:
movies_df = pd.read_csv('tmdb_5000_movies.csv')
movies_df.head()

,budget,genres,id,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",10-12-2009,2787965087,162.0,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",19-05-2007,961000000,169.0,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",26-10-2015,880674609,148.0,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",16-07-2012,1084939099,165.0,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",07-03-2012,284139100,132.0,"Lost in our world, found in another.",John Carter,6.1,2124


## Creating list of dictionaries 

We create a list of dictionaries such as genres, production comapnies, cast etc to link it with the main 'movies' table. 

In [3]:
#Creating a list of dictionaries 
genre_column_list =[]
for each in movies_df['genres']:
    genre_column_list.append(json.loads(each))

In [4]:
genre_column_list
movieid_genre_list = []
for i,x in enumerate(genre_column_list):
    for y in x:
        movieid_genre_list.append(movies_df['id'][i])
movieid_genre_list[1:5]

[19995, 19995, 19995, 285]

In [5]:
genre_list = []
genre_id_list=[]
for x in genre_column_list:
    for y in x:
        genre_id_list.append(y['id'])
        genre_list.append(y['name'])
            

In [6]:
genre_list
genre_df = pd.DataFrame(genre_list, columns=['Genre'])
genre_df['ID'] = pd.Series(genre_id_list)
genre_df['Movie_ID'] = pd.Series(movieid_genre_list)
genre_df = genre_df[['Movie_ID','ID', 'Genre']]

In [7]:
genre_df

,Movie_ID,ID,Genre
0,19995,28,Action
1,19995,12,Adventure
2,19995,14,Fantasy
3,19995,878,Science Fiction
4,285,12,Adventure
5,285,14,Fantasy
6,285,28,Action
7,206647,28,Action
8,206647,12,Adventure
9,206647,80,Crime


In [8]:
productioncomp_column_list = []
for each in movies_df['production_companies']:
    productioncomp_column_list.append(json.loads(each))
productioncomp_column_list

[[{'id': 289, 'name': 'Ingenious Film Partners'},
  {'id': 306, 'name': 'Twentieth Century Fox Film Corporation'},
  {'id': 444, 'name': 'Dune Entertainment'},
  {'id': 574, 'name': 'Lightstorm Entertainment'}],
 [{'id': 2, 'name': 'Walt Disney Pictures'},
  {'id': 130, 'name': 'Jerry Bruckheimer Films'},
  {'id': 19936, 'name': 'Second Mate Productions'}],
 [{'id': 5, 'name': 'Columbia Pictures'},
  {'id': 10761, 'name': 'Danjaq'},
  {'id': 69434, 'name': 'B24'}],
 [{'id': 923, 'name': 'Legendary Pictures'},
  {'id': 6194, 'name': 'Warner Bros.'},
  {'id': 9993, 'name': 'DC Entertainment'},
  {'id': 9996, 'name': 'Syncopy'}],
 [{'id': 2, 'name': 'Walt Disney Pictures'}],
 [{'id': 5, 'name': 'Columbia Pictures'},
  {'id': 326, 'name': 'Laura Ziskin Productions'},
  {'id': 19551, 'name': 'Marvel Enterprises'}],
 [{'id': 2, 'name': 'Walt Disney Pictures'},
  {'id': 6125, 'name': 'Walt Disney Animation Studios'}],
 [{'id': 420, 'name': 'Marvel Studios'},
  {'id': 15357, 'name': 'Prime Foc

In [9]:
company_id_list = []
company_list=[]
for x in productioncomp_column_list:
    for y in x:
        company_id_list.append(y['id'])
        company_list.append(y['name'])

In [12]:
movieid_prodcomp_list = []
for i, x in enumerate(productioncomp_column_list):
    for y in x:
        movieid_prodcomp_list.append(movies_df['id'][i])
movieid_prodcomp_list[:10]

[19995, 19995, 19995, 19995, 285, 285, 285, 206647, 206647, 206647]

## Converting these lists into 'Dataframes'

Once these lists are created, we create dataframes such as Genres, Movies, Cast, Production Companies for easily converting them into SQL tables.

In [13]:
prod_companies_df = pd.DataFrame(movieid_prodcomp_list, columns = ['Movie_ID'])
prod_companies_df['ID'] = pd.Series(company_id_list)
prod_companies_df['Company'] = pd.Series(company_list)
prod_companies_df[:10]

,Movie_ID,ID,Company
0,19995,289,Ingenious Film Partners
1,19995,306,Twentieth Century Fox Film Corporation
2,19995,444,Dune Entertainment
3,19995,574,Lightstorm Entertainment
4,285,2,Walt Disney Pictures
5,285,130,Jerry Bruckheimer Films
6,285,19936,Second Mate Productions
7,206647,5,Columbia Pictures
8,206647,10761,Danjaq
9,206647,69434,B24


## Reading the second database 'CAST'

This table consisted of multiple values and hence was not atomic. Inorder to make sure all our tables are in the normal form, the following chunk of code is used for data preprocessing and making it atomic.

In [14]:
cast_xl_df = pd.read_csv('tmdb_5000_cast.csv')
cast_xl_df.head()

,movie_id,title,cast
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c..."


In [15]:
cast_column_list =[]
for each in cast_xl_df['cast'][:218]:
    cast_column_list.append(json.loads(each))

In [16]:
for each in cast_xl_df['cast'][219:598]:
    cast_column_list.append(json.loads(each))

In [17]:
for each in cast_xl_df['cast'][599:637]:
    cast_column_list.append(json.loads(each))

In [18]:
for each in cast_xl_df['cast'][638:4208]:
    cast_column_list.append(json.loads(each))

In [19]:
for each in cast_xl_df['cast'][4209:]:
    cast_column_list.append(json.loads(each))

In [21]:
final_cast_column =[]
for x in cast_column_list:
    dummy_column=[]
    for y in x[:5]:
        dummy_column.append(y)
    final_cast_column.append(dummy_column)
final_cast_column

[[{'cast_id': 242,
   'character': 'Jake Sully',
   'credit_id': '5602a8a7c3a3685532001c9a',
   'gender': 2,
   'id': 65731,
   'name': 'Sam Worthington',
   'order': 0},
  {'cast_id': 3,
   'character': 'Neytiri',
   'credit_id': '52fe48009251416c750ac9cb',
   'gender': 1,
   'id': 8691,
   'name': 'Zoe Saldana',
   'order': 1},
  {'cast_id': 25,
   'character': 'Dr. Grace Augustine',
   'credit_id': '52fe48009251416c750aca39',
   'gender': 1,
   'id': 10205,
   'name': 'Sigourney Weaver',
   'order': 2},
  {'cast_id': 4,
   'character': 'Col. Quaritch',
   'credit_id': '52fe48009251416c750ac9cf',
   'gender': 2,
   'id': 32747,
   'name': 'Stephen Lang',
   'order': 3},
  {'cast_id': 5,
   'character': 'Trudy Chacon',
   'credit_id': '52fe48009251416c750ac9d3',
   'gender': 1,
   'id': 17647,
   'name': 'Michelle Rodriguez',
   'order': 4}],
 [{'cast_id': 4,
   'character': 'Captain Jack Sparrow',
   'credit_id': '52fe4232c3a36847f800b50d',
   'gender': 2,
   'id': 85,
   'name': 'Jo

In [23]:
movieid_list = list(cast_xl_df['movie_id'])
for i in sorted([218,598,637,4208], reverse=True):
    del movieid_list[i]
len(movieid_list)

4757

In [24]:
movieid_cast_list=[]
for i, x in enumerate(final_cast_column):
    for y in x:
        movieid_cast_list.append(movieid_list[i])
len(movieid_cast_list)

23564

In [25]:
cast_id_list = []
cast_charname_list = []
cast_name_list =[]
for y in final_cast_column:
    for x in y:
        cast_id_list.append(x['id'])
        cast_charname_list.append(x['character'])
        cast_name_list.append(x['name'])

### Final CAST and MOVIES dataframe:

In [26]:
cast_df = pd.DataFrame(movieid_cast_list, columns=['Movie_ID'])
cast_df['ID'] = pd.Series(cast_id_list)
cast_df['Character'] = pd.Series(cast_charname_list)
cast_df['Actor'] = pd.Series(cast_name_list)
cast_df

,Movie_ID,ID,Character,Actor
0,19995,65731,Jake Sully,Sam Worthington
1,19995,8691,Neytiri,Zoe Saldana
2,19995,10205,Dr. Grace Augustine,Sigourney Weaver
3,19995,32747,Col. Quaritch,Stephen Lang
4,19995,17647,Trudy Chacon,Michelle Rodriguez
5,285,85,Captain Jack Sparrow,Johnny Depp
6,285,114,Will Turner,Orlando Bloom
7,285,116,Elizabeth Swann,Keira Knightley
8,285,1640,"William ""Bootstrap Bill"" Turner",Stellan Skarsgård
9,285,1619,Captain Sao Feng,Chow Yun-fat


In [27]:
movies_df=movies_df[['id','title','budget','revenue','original_language','tagline','overview','release_date','runtime','popularity','vote_average','vote_count']]
movies_df

,id,title,budget,revenue,original_language,tagline,overview,release_date,runtime,popularity,vote_average,vote_count
0,19995,Avatar,237000000,2787965087,en,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",10-12-2009,162.0,150.437577,7.2,11800
1,285,Pirates of the Caribbean: At World's End,300000000,961000000,en,"At the end of the world, the adventure begins.","Captain Barbossa, long believed to be dead, ha...",19-05-2007,169.0,139.082615,6.9,4500
2,206647,Spectre,245000000,880674609,en,A Plan No One Escapes,A cryptic message from Bond’s past sends him o...,26-10-2015,148.0,107.376788,6.3,4466
3,49026,The Dark Knight Rises,250000000,1084939099,en,The Legend Ends,Following the death of District Attorney Harve...,16-07-2012,165.0,112.312950,7.6,9106
4,49529,John Carter,260000000,284139100,en,"Lost in our world, found in another.","John Carter is a war-weary, former military ca...",07-03-2012,132.0,43.926995,6.1,2124
5,559,Spider-Man 3,258000000,890871626,en,The battle within.,The seemingly invincible Spider-Man goes up ag...,01-05-2007,139.0,115.699814,5.9,3576
6,38757,Tangled,260000000,591794936,en,They're taking adventure to new lengths.,When the kingdom's most wanted-and most charmi...,24-11-2010,100.0,48.681969,7.4,3330
7,99861,Avengers: Age of Ultron,280000000,1405403694,en,A New Age Has Come.,When Tony Stark tries to jumpstart a dormant p...,22-04-2015,141.0,134.279229,7.3,6767
8,767,Harry Potter and the Half-Blood Prince,250000000,933959197,en,Dark Secrets Revealed,"As Harry begins his sixth year at Hogwarts, he...",07-07-2009,153.0,98.885637,7.4,5293
9,209112,Batman v Superman: Dawn of Justice,250000000,873260194,en,Justice or revenge,Fearing the actions of a god-like Super Hero l...,23-03-2016,151.0,155.790452,5.7,7004


In [28]:
movies_xl_df=pd.read_csv('tmdb_5000_movies.csv')


countries_column_list =[]
for each in movies_xl_df['production_countries']:
    countries_column_list.append(json.loads(each))
    
    
movieid_countries_list = []
for i,x in enumerate(countries_column_list):
    for y in x:
        movieid_countries_list.append(movies_xl_df['id'][i])

countries_name_list=[]
for x in countries_column_list:
    for y in x:
        countries_name_list.append(y['name'])

countries_df = pd.DataFrame(countries_name_list, columns=['Shoot_Location'])
countries_df['Movie_ID'] = pd.Series(movieid_countries_list)
countries_df = countries_df[['Movie_ID', 'Shoot_Location']]
countries_df.head()

,Movie_ID,Shoot_Location
0,19995,United States of America
1,19995,United Kingdom
2,285,United States of America
3,206647,United Kingdom
4,206647,United States of America


In [29]:
movieid_abv_7 = movies_df[movies_df['vote_average'] >= 7]['id']
movieid_abv_7 = list(movieid_abv_7)
movieid_abv_7

[19995,
 49026,
 38757,
 99861,
 767,
 58,
 24428,
 122917,
 57158,
 597,
 271110,
 62211,
 10193,
 168259,
 127585,
 54138,
 64682,
 10681,
 155,
 14160,
 1726,
 44826,
 137113,
 150540,
 102651,
 119450,
 100402,
 177572,
 82690,
 10191,
 118340,
 157336,
 27205,
 49051,
 4922,
 49538,
 675,
 674,
 2062,
 272,
 109445,
 76341,
 177677,
 81188,
 616,
 13475,
 82702,
 13183,
 281957,
 2503,
 102899,
 101299,
 673,
 671,
 22,
 51497,
 58574,
 861,
 87827,
 1895,
 585,
 49047,
 2567,
 120,
 286217,
 98,
 180,
 672,
 36557,
 280,
 4982,
 68718,
 11688,
 1372,
 106646,
 9016,
 29193,
 3131,
 76758,
 855,
 18,
 10501,
 12,
 122,
 121,
 61791,
 9806,
 863,
 116745,
 1422,
 10674,
 65754,
 10528,
 686,
 921,
 8358,
 2502,
 161,
 853,
 1904,
 16858,
 22538,
 1949,
 11544,
 11324,
 4147,
 228150,
 9023,
 161795,
 49948,
 36970,
 453,
 8587,
 309809,
 93456,
 9693,
 75656,
 594,
 203801,
 197,
 16869,
 587,
 57212,
 2756,
 2270,
 978,
 20352,
 9008,
 857,
 425,
 603,
 568,
 82695,
 227306,
 345,

In [30]:
actors_abv_7 = []
for i, x in enumerate(cast_df['Movie_ID']):
    if x in movieid_abv_7:
        actors_abv_7.append(cast_df['Actor'][i])
actors_abv_7 = list(set(actors_abv_7))
indices = [55,122,116,117,118]
for i in sorted(indices, reverse=True):
    del actors_abv_7[i]
len(actors_abv_7)

3096

# Scraping Twitter data using the Python-Twitter API 

In [31]:
import urllib
import oauth2 as oauth

consumer_key = 'zYBUuTWO3nnm8Rmi5HLKKANV8'
consumer_secret = 'kifuFCVkDXthnHwbEni92pKfowa32VTF1l3C6cliwfqFFue9qw'

request_token_url = 'https://api.twitter.com/oauth/request_token'
access_token_url = 'https://api.twitter.com/oauth/access_token'
authorize_url = 'https://api.twitter.com/oauth/authorize'

consumer = oauth.Consumer(consumer_key, consumer_secret)
client = oauth.Client(consumer)

# Step 1: Get a request token. This is a temporary token that is used for 
# having the user authorize an access token and to sign the request to obtain 
# said access token.
    
resp, content = client.request(request_token_url, "GET")
if resp['status'] != '200':
    raise Exception("Invalid response %s." % resp['status'])
request_token = dict(urllib.parse.parse_qsl(content))
print(request_token)
# print(request_token[b'oauth_token'])

print("Request Token:")
print("    - oauth_token        = %s" % request_token[b'oauth_token'])
print("    - oauth_token_secret = %s" % request_token[b'oauth_token_secret'])
print 

# Step 2: Redirect to the provider. Since this is a CLI script we do not 
# redirect. In a web application you would redirect the user to the URL
# below.

print("Go to the following link in your browser:")
print("%s?oauth_token=%s" % (authorize_url, request_token[b'oauth_token'].decode('ascii')))
print 

# After the user has granted access to you, the consumer, the provider will
# redirect you to whatever URL you have told them to redirect to. You can 
# usually define this in the oauth_callback argument as well.
accepted = 'n'
while accepted.lower() == 'n':
    accepted = input('Have you authorized me? (y/n) ')
oauth_verifier = input('What is the PIN? ')

# Step 3: Once the consumer has redirected the user back to the oauth_callback
# URL you can request the access token the user has approved. You use the 
# request token to sign this request. After this is done you throw away the
# request token and use the access token returned. You should store this 
# access token somewhere safe, like a database, for future use.
token = oauth.Token(request_token[b'oauth_token'],
    request_token[b'oauth_token_secret'])
token.set_verifier(oauth_verifier)
client = oauth.Client(consumer, token)

resp, content = client.request(access_token_url, "POST")
access_token = dict(urllib.parse.parse_qsl(content))
print(access_token)
print ("Access Token:")
print ("    - oauth_token        = %s" % access_token[b'oauth_token'].decode('ascii'))
print ("    - oauth_token_secret = %s" % access_token[b'oauth_token_secret'].decode('ascii'))
print
print ("You may now access protected resources using the access tokens above." )
print

{b'oauth_token': b's7PhPwAAAAAA4nyUAAABYtGiSW4', b'oauth_token_secret': b'SaN59MBrC55IaahVOBKsb55nX9admGQc', b'oauth_callback_confirmed': b'true'}
Request Token:
    - oauth_token        = b's7PhPwAAAAAA4nyUAAABYtGiSW4'
    - oauth_token_secret = b'SaN59MBrC55IaahVOBKsb55nX9admGQc'
Go to the following link in your browser:
https://api.twitter.com/oauth/authorize?oauth_token=s7PhPwAAAAAA4nyUAAABYtGiSW4
Have you authorized me? (y/n) y
What is the PIN? 9489933
{b'oauth_token': b'967491818291113990-fHpYS11pQdeb0KMVAoXC4kN3xwRTGK5', b'oauth_token_secret': b'duv0MIeIXqegdfYsbgdRjJgUFkvz27WFgQ8kzq3IRpuTU', b'user_id': b'967491818291113990', b'screen_name': b'HamzaHasher'}
Access Token:
    - oauth_token        = 967491818291113990-fHpYS11pQdeb0KMVAoXC4kN3xwRTGK5
    - oauth_token_secret = duv0MIeIXqegdfYsbgdRjJgUFkvz27WFgQ8kzq3IRpuTU
You may now access protected resources using the access tokens above.


<function print>

In [32]:
import twitter
import json

consumer_key = "zYBUuTWO3nnm8Rmi5HLKKANV8"
consumer_secret = "kifuFCVkDXthnHwbEni92pKfowa32VTF1l3C6cliwfqFFue9qw"
access_token = "967491818291113990-fHpYS11pQdeb0KMVAoXC4kN3xwRTGK5"
access_token_secret = "duv0MIeIXqegdfYsbgdRjJgUFkvz27WFgQ8kzq3IRpuTU"

api = twitter.Api(consumer_key=consumer_key,
                  consumer_secret=consumer_secret,
                  access_token_key=access_token,
                  access_token_secret=access_token_secret)

In [33]:
actor_tags=[]   #Scraping twitter public data to see what the public is saying about the actors
users=[]
# Twitter allows only 180 tweet searches per 15 minutes, so chunking is done
for each in actors_abv_7[:180]:
    each = each.replace("'",' ')
    first_and_last_name = each.split(" ")    # Splitting each actors name into first, middle and last name
    first_name = first_and_last_name[0]
    last_name=''
    middle_name=''
    if len(first_and_last_name) == 2:
        last_name = first_and_last_name[1]
    if len(first_and_last_name) == 3:
        middle_name = first_and_last_name[1]
        last_name = first_and_last_name[2]
    
    results = api.GetSearch(raw_query="q=" +first_name+"%20"+middle_name+"%20"+last_name+ "&count=100")
    cnt = len(results)
    #print(cnt)              #Making a call for each actor on twitter
    for res in results:
        x = json.loads(str(res))
        actor_info={
            'name' : first_name + ' ' + last_name,
            'tags' : [],
            'count' : cnt
        }
        user_info={
            'id': '',
            'name':''
        }
        user_info['id'] = x['user']['id']
        user_info['name'] = x['user']['name']
        users.append(user_info)
        for item in x['hashtags']:
            actor_info['tags'].append(item['text'])
    actor_tags.append(actor_info)

In [34]:
users = users[:500]
users

[{'id': 379383657, 'name': 'T Slagin'},
 {'id': 228554896, 'name': 'Michal Udriela'},
 {'id': 239546374, 'name': 'Bridget61'},
 {'id': 897816785067495424, 'name': 'Image Quotes'},
 {'id': 1177164313, 'name': 'Jesus of Suburbia'},
 {'id': 4072190103, 'name': 'Dean Corwin'},
 {'id': 1871348034, 'name': 'Deb'},
 {'id': 606713765, 'name': 'Luanne Nelson'},
 {'id': 976268485717348352, 'name': 'Daylight Snobbery'},
 {'id': 83450413, 'name': 'Name'},
 {'id': 83450413, 'name': 'Name'},
 {'id': 244115238, 'name': 'Carlos Gonzalez'},
 {'id': 27991531, 'name': 'babadook spinoza ☭'},
 {'id': 6480442, 'name': 'Matt Martinez'},
 {'id': 27991531, 'name': 'babadook spinoza ☭'},
 {'id': 965123497134559232, 'name': 'Nelson Luis Monier Quevedo'},
 {'id': 3571808356, 'name': 'Trump-Pence USA'},
 {'id': 141922590, 'name': 'Robb S.'},
 {'id': 151151350, 'name': 'nelson juarez'},
 {'id': 805294198588964864, 'name': 'Nelson Goins'},
 {'id': 1357539090, 'name': 'Ryan Harrold'},
 {'id': 704145840588193793, 'nam

In [35]:
%store -r tags_df

In [39]:
results = api.GetUserTimeline(screen_name='AnupamPKher', count=100) #Storing Anupam Khers Twitter Data
posts =[]
favourites =[]
retweets = []
time = []
user_id = []
name = []
followers = []
hashtags = []
for each in results:
    status = json.loads(str(each))
    posts.append(status['text'])
    try:
        favourites.append(status['favorite_count'])
    except:
        favourites.append(status['retweeted_status']['favorite_count'])
    retweets.append(status['retweet_count'])
    time.append(status['created_at'])
    user_id.append(status['user']['id'])
    followers.append(status['user']['followers_count'])
    name.append(status['user']['name'])
    try:
        hashtags.append(status['hashtags'][0]['text'])
    except:
        hashtags.append(status['hashtags'])

In [40]:
df_1 = pd.DataFrame(data = name, columns = ['Name'])
df_1['User_id'] = user_id
df_1['Followers'] = followers
df_1['Post'] = posts
df_1['Hashtags'] = hashtags
df_1['Posted_time'] = time
df_1['Favourites'] = favourites
df_1['Retweets'] = retweets
df_1

,Name,User_id,Followers,Post,Hashtags,Posted_time,Favourites,Retweets
0,Anupam Kher,76294950,12386995,"The best remedy for those who are afraid, lone...",[],Mon Apr 16 19:07:49 +0000 2018,896,55
1,Anupam Kher,76294950,12386995,Superb.👍😊 https://t.co/xiTUnk4uLi,[],Mon Apr 16 13:45:22 +0000 2018,506,17
2,Anupam Kher,76294950,12386995,That is Great. Congratulations @partinator @Sa...,TheBoyWithTheTopKnot,Mon Apr 16 11:05:41 +0000 2018,217,11
3,Anupam Kher,76294950,12386995,Sure!! Will mail you. Thanks.:) https://t.co/H...,[],Mon Apr 16 11:01:52 +0000 2018,295,14
4,Anupam Kher,76294950,12386995,Thank you.🙏 https://t.co/7A4ET0otvM,[],Mon Apr 16 10:18:22 +0000 2018,299,17
5,Anupam Kher,76294950,12386995,RT @indiaforums: .@AnupamPKher bags the 6th po...,[],Mon Apr 16 10:17:35 +0000 2018,498,41
6,Anupam Kher,76294950,12386995,RT @Koimoi: .@AnupamPKher ranks 6th on #Richto...,Richtopia,Mon Apr 16 10:16:41 +0000 2018,256,16
7,Anupam Kher,76294950,12386995,Wow.🤓🤓🤓. This is FANTASTIC!! 😊. Thank you @Ric...,[],Mon Apr 16 09:56:48 +0000 2018,375,37
8,Anupam Kher,76294950,12386995,I have great memories of #RoopKiRaniChoronKaRa...,RoopKiRaniChoronKaRaja,Mon Apr 16 08:15:25 +0000 2018,979,91
9,Anupam Kher,76294950,12386995,Congratulations my dearest @priyankakher79 fo...,[],Mon Apr 16 07:54:12 +0000 2018,736,65


In [41]:
results = api.GetUserTimeline(screen_name='TheRock', count=100) #Storing Dwayne Johnsons Twitter data into a dataframe
posts =[]
favourites =[]
retweets = []
time = []
user_id = []
name = []
followers = []
hashtags = []
for each in results:
    status = json.loads(str(each))
    posts.append(status['text'])
    try:
        favourites.append(status['favorite_count'])
    except:
        favourites.append(status['retweeted_status']['favorite_count'])
    retweets.append(status['retweet_count'])
    time.append(status['created_at'])
    user_id.append(status['user']['id'])
    followers.append(status['user']['followers_count'])
    name.append(status['user']['name'])
    try:
        hashtags.append(status['hashtags'][0]['text'])
    except:
        hashtags.append(status['hashtags'])
df_2 = pd.DataFrame(data = name, columns = ['Name'])
df_2['User_id'] = user_id
df_2['Followers'] = followers
df_2['Post'] = posts
df_2['Hashtags'] = hashtags
df_2['Posted_time'] = time
df_2['Favourites'] = favourites
df_2['Retweets'] = retweets
df_2

,Name,User_id,Followers,Post,Hashtags,Posted_time,Favourites,Retweets
0,Dwayne Johnson,250831586,12827195,You must. We all must. \nGood to see you motiv...,ProjectRock,Mon Apr 16 18:26:55 +0000 2018,1433,129
1,Dwayne Johnson,250831586,12827195,Jeff Bezos + RAMPAGE = 🤙🏾\nThe love and respec...,[],Mon Apr 16 17:57:24 +0000 2018,2179,165
2,Dwayne Johnson,250831586,12827195,@scarlettwitch7 Thx so much for the support!,[],Mon Apr 16 08:05:54 +0000 2018,33,6
3,Dwayne Johnson,250831586,12827195,Strong weekend of RAMPAGE business bringing in...,[],Mon Apr 16 07:36:32 +0000 2018,3611,331
4,Dwayne Johnson,250831586,12827195,RT @VictoriaShianne: Y’all... go see Rampage! ...,[],Mon Apr 16 06:05:27 +0000 2018,868,52
5,Dwayne Johnson,250831586,12827195,Thx dude appreciate you loving RAMPAGE. I’m do...,[],Mon Apr 16 04:58:15 +0000 2018,3433,220
6,Dwayne Johnson,250831586,12827195,@The305MVP Haha your little dude already swing...,[],Mon Apr 16 04:45:18 +0000 2018,346,20
7,Dwayne Johnson,250831586,12827195,Thx dude glad you and pops had a cool time at ...,[],Mon Apr 16 03:12:35 +0000 2018,1651,123
8,Dwayne Johnson,250831586,12827195,Haha brother the love and admiration is mutual...,[],Mon Apr 16 01:33:20 +0000 2018,2294,172
9,Dwayne Johnson,250831586,12827195,THANK U so much everyone ‘round the 🌎 for maki...,[],Mon Apr 16 01:06:07 +0000 2018,6113,558


In [53]:
results = api.GetUserTimeline(screen_name='mrdannyglover', count=100) #Adding Danny Glovers twitter data
posts =[]
favourites =[]
retweets = []
time = []
user_id = []
name = []
followers = []
hashtags = []
for each in results:
    status = json.loads(str(each))
    posts.append(status['text'])
    try:
        favourites.append(status['favorite_count'])
    except:
        favourites.append(status['retweeted_status']['favorite_count'])
    try:
        retweets.append(status['retweet_count'])
    except:
        retweets.append(0)
    time.append(status['created_at'])
    user_id.append(status['user']['id'])
    followers.append(status['user']['followers_count'])
    name.append(status['user']['name'])
    try:
        hashtags.append(status['hashtags'][0]['text'])
    except:
        hashtags.append(status['hashtags'])
df_3 = pd.DataFrame(data = name, columns = ['Name'])
df_3['User_id'] = user_id
df_3['Followers'] = followers
df_3['Post'] = posts
df_3['Hashtags'] = hashtags
df_3['Posted_time'] = time
df_3['Favourites'] = favourites
df_3['Retweets'] = retweets
df_3

,Name,User_id,Followers,Post,Hashtags,Posted_time,Favourites,Retweets
0,Danny Glover,32309662,1466022,"RT @BWOPATILE: Actor, Producer, Humanitarian @...",[],Mon Apr 16 22:48:37 +0000 2018,10,4
1,Danny Glover,32309662,1466022,New Poll Shows Pamela Price Leading Nancy O’Ma...,[],Thu Apr 12 19:15:16 +0000 2018,27,7
2,Danny Glover,32309662,1466022,"Sundance Winner ‘Hale County This Morning, Thi...",[],Wed Apr 11 19:08:28 +0000 2018,27,9
3,Danny Glover,32309662,1466022,RT @TheRealNews: During a Q&amp;A with actor @...,[],Wed Apr 11 15:42:44 +0000 2018,98,64
4,Danny Glover,32309662,1466022,RT @MarkWeisbrot: Great statement from Ro Khan...,[],Tue Apr 10 23:48:03 +0000 2018,15,9
5,Danny Glover,32309662,1466022,RT @TheSandersInst: Why do we vote on a Tuesda...,[],Tue Apr 10 22:43:43 +0000 2018,762,432
6,Danny Glover,32309662,1466022,RT @RepRoKhanna: I am very concerned that the ...,[],Sat Apr 07 21:59:17 +0000 2018,1076,402
7,Danny Glover,32309662,1466022,"RT @I_AM_2018: ""I hope and pray that this gath...",[],Sat Apr 07 00:24:31 +0000 2018,44,16
8,Danny Glover,32309662,1466022,RT @MajorCBS: A discussion on political activi...,MLK50,Fri Apr 06 16:03:22 +0000 2018,16,14
9,Danny Glover,32309662,1466022,"RT @AFSCME: ""We gotta do what we do best - org...",[],Fri Apr 06 16:02:05 +0000 2018,41,26


In [46]:
results = api.GetUserTimeline(screen_name='JanetJackson', count=100) #Adding Janet Jacksons twitter data
posts =[]
favourites =[]
retweets = []
time = []
user_id = []
name = []
followers = []
hashtags = []
for each in results:
    status = json.loads(str(each))
    posts.append(status['text'])
    try:
        favourites.append(status['favorite_count'])
    except:
        favourites.append(status['retweeted_status']['favorite_count'])
    try:
        retweets.append(status['retweet_count'])
    except:
        retweets.append(0)
    time.append(status['created_at'])
    user_id.append(status['user']['id'])
    followers.append(status['user']['followers_count'])
    name.append(status['user']['name'])
    try:
        hashtags.append(status['hashtags'][0]['text'])
    except:
        hashtags.append(status['hashtags'])
df_4 = pd.DataFrame(data = name, columns = ['Name'])
df_4['User_id'] = user_id
df_4['Followers'] = followers
df_4['Post'] = posts
df_4['Hashtags'] = hashtags
df_4['Posted_time'] = time
df_4['Favourites'] = favourites
df_4['Retweets'] = retweets
df_4

,Name,User_id,Followers,Post,Hashtags,Posted_time,Favourites,Retweets
0,Janet Jackson,52939106,3464515,Attention... it’s time to #DANCEWITHJANET! Sho...,DANCEWITHJANET,Thu Apr 12 16:13:23 +0000 2018,6858,2330
1,Janet Jackson,52939106,3464515,"Dr. #MayaAngelou, the original #PhenomenalWoma...",MayaAngelou,Wed Apr 04 17:37:40 +0000 2018,6076,1924
2,Janet Jackson,52939106,3464515,So happy we met and so proud of all that you’r...,[],Wed Mar 21 17:25:01 +0000 2018,7784,1311
3,Janet Jackson,52939106,3464515,So sweet you are ❤️ https://t.co/NxRaQrFuvJ,[],Thu Mar 15 17:46:21 +0000 2018,10989,1937
4,Janet Jackson,52939106,3464515,How amazing is this? \nWhat a talent! https://...,[],Thu Mar 15 17:03:22 +0000 2018,3712,921
5,Janet Jackson,52939106,3464515,I had the best time with the kids at @MPA_ACAD...,[],Mon Mar 12 16:01:32 +0000 2018,2898,566
6,Janet Jackson,52939106,3464515,"Our time has come to rejoice, a new agenda’s d...",Equality,Thu Mar 08 18:07:16 +0000 2018,20922,3982
7,Janet Jackson,52939106,3464515,https://t.co/mdgrlmQV5D https://t.co/TzRscRzBWc,[],Wed Jan 31 21:46:37 +0000 2018,3284,918
8,Janet Jackson,52939106,3464515,https://t.co/XftiEXb0ZY https://t.co/zsxH1tB6Zy,[],Tue Jan 30 03:41:00 +0000 2018,7136,2106
9,Janet Jackson,52939106,3464515,Love you sis! ❤️ https://t.co/9fK8Br48hD,[],Fri Jan 26 22:21:34 +0000 2018,7237,1377


In [339]:
twitter_data_df = pd.concat([df_1,df_2,df_3,df_4])
twitter_data_df

,Name,User_id,Followers,Post,Hashtags,Posted_time,Favourites,Retweets
0,Anupam Kher,76294950,12386995,"The best remedy for those who are afraid, lone...",[],Mon Apr 16 19:07:49 +0000 2018,896,55
1,Anupam Kher,76294950,12386995,Superb.👍😊 https://t.co/xiTUnk4uLi,[],Mon Apr 16 13:45:22 +0000 2018,506,17
2,Anupam Kher,76294950,12386995,That is Great. Congratulations @partinator @Sa...,TheBoyWithTheTopKnot,Mon Apr 16 11:05:41 +0000 2018,217,11
3,Anupam Kher,76294950,12386995,Sure!! Will mail you. Thanks.:) https://t.co/H...,[],Mon Apr 16 11:01:52 +0000 2018,295,14
4,Anupam Kher,76294950,12386995,Thank you.🙏 https://t.co/7A4ET0otvM,[],Mon Apr 16 10:18:22 +0000 2018,299,17
5,Anupam Kher,76294950,12386995,RT @indiaforums: .@AnupamPKher bags the 6th po...,[],Mon Apr 16 10:17:35 +0000 2018,498,41
6,Anupam Kher,76294950,12386995,RT @Koimoi: .@AnupamPKher ranks 6th on #Richto...,Richtopia,Mon Apr 16 10:16:41 +0000 2018,256,16
7,Anupam Kher,76294950,12386995,Wow.🤓🤓🤓. This is FANTASTIC!! 😊. Thank you @Ric...,[],Mon Apr 16 09:56:48 +0000 2018,375,37
8,Anupam Kher,76294950,12386995,I have great memories of #RoopKiRaniChoronKaRa...,RoopKiRaniChoronKaRaja,Mon Apr 16 08:15:25 +0000 2018,979,91
9,Anupam Kher,76294950,12386995,Congratulations my dearest @priyankakher79 fo...,[],Mon Apr 16 07:54:12 +0000 2018,736,65


In [341]:
for i,x in enumerate(twitter_data_df['Hashtags']):
    if len(x) == 0:
        twitter_data_df['Hashtags'].iloc[i] = 'N/A'

C:\Users\hashe\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [407]:
public_posts =[]
for i,x in enumerate(api.GetSearch(term='MayaAngelou')):
    public_tweet=dict()
    public_tweet['name'] = json.loads(str(x))['user']['name']
    public_tweet['id'] = json.loads(str(x))['id']
    public_tweet['followers'] = json.loads(str(x))['user']['followers_count']
    public_tweet['post'] = json.loads(str(x))['text']
    public_tweet['favourites'] = json.loads(str(x))['user']['favourites_count']
    public_posts.append(public_tweet)
public_posts

[{'favourites': 13286,
  'followers': 21592,
  'id': 989081335816957952,
  'name': 'AntiBullyingAlliance',
  'post': "'If you're always trying to be 'normal', you will never know how amazing you can be' - #mayaangelou \n\n#antibullying… https://t.co/lmtsfqrZEv"},
 {'favourites': 8,
  'followers': 614908,
  'id': 987004905700421632,
  'name': 'Maya Angelou',
  'post': '"One isn’t necessarily born with courage, but one is born with potential. Without courage, we cannot practice any o… https://t.co/GoOySQZ0xb'},
 {'favourites': 8,
  'followers': 614908,
  'id': 988117547382919170,
  'name': 'Maya Angelou',
  'post': '“I had a lot of clouds, but I have had so many rainbows.” #MayaAngelou Dr. Angelou taught us to count our blessings… https://t.co/aBhYprd7jq'},
 {'favourites': 314,
  'followers': 1593,
  'id': 989344125802696704,
  'name': 'Valerie Lewis',
  'post': "Defeat isn't the end, it's an opportunity for correction, for learning. It's only the end when you quit.😉 #Quote… https://t.co

In [409]:
public_tweets_df = pd.DataFrame(public_posts)
public_tweets_df = public_tweets_df[['id','name','post','followers']]
public_tweets_df

,id,name,post,followers
0,989081335816957952,AntiBullyingAlliance,"'If you're always trying to be 'normal', you w...",21592
1,987004905700421632,Maya Angelou,"""One isn’t necessarily born with courage, but ...",614908
2,988117547382919170,Maya Angelou,"“I had a lot of clouds, but I have had so many...",614908
3,989344125802696704,Valerie Lewis,"Defeat isn't the end, it's an opportunity for ...",1593
4,989339706038063104,The JoP ♥ ✌️,RT @ABAonline: 'If you're always trying to be ...,2307
5,989338213411106817,Roberta Thompson,Another night at the hospital with my sister a...,1089
6,989334454673002496,OmahaChildren'sChoir,Our final post for National Poetry Month is Ma...,492
7,989331830846476288,Husky Haddish,"#Kanye ""When someone shows you who they are, b...",2238
8,989331237620928514,Seth Pedone,#MayaAngelou #stillirise #positivity #strength...,24
9,989314558090235904,Douglas Connors,THIS here is why #ILoveDance\n\nhttps://t.co/w...,1632


# Scraping Instagram tags data using the Instagram-Python API

In [800]:
import requests
import urllib.request
import json
APP_ACCESS_TOKEN='51285080.a87c0a5.dac5080732934beaa634253b7b440052'

In [801]:
#genres:actionmovies, horror etc
#actors:dwaynejohnson,bradpitt etc


url1="https://api.instagram.com/v1/tags/search?q=actionmovies&access_token=%s" %(APP_ACCESS_TOKEN)
f = urllib.request.urlopen(url1)
myfile = f.read()
x=json.loads(myfile)

count_x=[]
for each in x['data']:
    count_x.append(each['media_count'])
    
count_x
name_x=[]
for each in x['data']:
    name_x.append(each['name'])
name_x
count_x


act_url="https://api.instagram.com/v1/tags/search?q=dwaynejohnson&access_token=%s" %(APP_ACCESS_TOKEN)
act_f = urllib.request.urlopen(act_url)
act_myfile = act_f.read()
act_x=json.loads(act_myfile)


act_count=[]
for each in act_x['data']:
    act_count.append(each['media_count'])
    
act_count
act_name=[]
for each in act_x['data']:
    act_name.append(each['name'])


In [803]:
url2="https://api.instagram.com/v1/tags/search?q=horror&access_token=%s" %(APP_ACCESS_TOKEN)
f = urllib.request.urlopen(url2)
myfile1 = f.read()
x1=json.loads(myfile1)

count_x1=[]
for each in x1['data']:
    count_x1.append(each['media_count'])
    
count_x1
name_x1=[]
for each in x1['data']:
    name_x1.append(each['name'])
name_x1


act_url1="https://api.instagram.com/v1/tags/search?q=bradpitt&access_token=%s" %(APP_ACCESS_TOKEN)
act_f1 = urllib.request.urlopen(act_url1)
act_myfile1 = act_f1.read()
act_x1=json.loads(act_myfile1)


act_count1=[]
for each in act_x1['data']:
    act_count1.append(each['media_count'])
    
act_count1
act_name1=[]
for each in act_x1['data']:
    act_name1.append(each['name'])
act_name1

['bradpitt',
 'bradpittfans',
 'bradpitted',
 'bradpittdaily',
 'bradpittscousin',
 'bradpittofdogs',
 'bradpitto',
 'bradpitty',
 'bradpitts',
 'bradpittfan',
 'bradpittlover',
 'bradpittlookalike',
 'bradpittbull',
 'bradpittong',
 'bradpittandangelinajolie',
 'bradpittcanino',
 'bradpittart',
 'bradpittdacarsalesman',
 'bradpittedit',
 'bradpittofcats',
 'bradpittwho',
 'bradpitttattoo',
 'bradpittlatino',
 'bradpitthair',
 'bradpittismycousin',
 'bradpittforever',
 'bradpittthatsmycousin',
 'bradpittmovie',
 'bradpittangelinajolie',
 'bradpittofkitties',
 'bradpittthedog',
 'bradpittmovies',
 'bradpittstyle',
 'bradpitthamster',
 'bradpittfury',
 'bradpitt_iran',
 'bradpittyoung',
 'bradpittdrawing',
 'bradpittwashere',
 'bradpittofficial',
 'bradpittseinsohn',
 'bradpittrp',
 'bradpittbeforegrabclit']

In [804]:
url3="https://api.instagram.com/v1/tags/search?q=romance&access_token=%s" %(APP_ACCESS_TOKEN)
f = urllib.request.urlopen(url3)
myfile2 = f.read()
x2=json.loads(myfile2)

count_x2=[]
for each in x2['data']:
    count_x2.append(each['media_count'])
    
name_x2=[]
for each in x2['data']:
    name_x2.append(each['name'])


act_url2="https://api.instagram.com/v1/tags/search?q=johnnydepp&access_token=%s" %(APP_ACCESS_TOKEN)
act_f2 = urllib.request.urlopen(act_url2)
act_myfile2 = act_f2.read()
act_x2=json.loads(act_myfile2)


act_count2=[]
for each in act_x2['data']:
    act_count2.append(each['media_count'])
    
act_count2
act_name2=[]
for each in act_x2['data']:
    act_name2.append(each['name'])
act_name2

['johnnydepp',
 'johnnydeppfans',
 'johnnydeppforever',
 'johnnydeppfan',
 'johnnydeppfanpage',
 'johnnydeppmovies',
 'johnnydepplookalike',
 'johnnydeppyoung',
 'johnnydepplove',
 'johnnydeppisinnocent',
 'johnnydepplovers',
 'johnnydeppquotes',
 'johnnydeppstyle',
 'johnnydeppneedsaoscar',
 'johnnydeppisperfect',
 'johnnydeppfandom',
 'johnnydeppedit',
 'johnnydeppishot',
 'johnnydeppislife',
 'johnnydeppily',
 'johnnydepppictures',
 'johnnydeppforlife',
 'johnnydepprp',
 'johnnydepphasourtrust',
 'johnnydeppmylove',
 'johnnydeppphotoshoot',
 'johnnydeppcosplay',
 'johnnydeppweloveyou',
 'johnnydepplover',
 'johnnydeppquote',
 'johnnydepptattoo',
 'johnnydepprare',
 'johnnydeppismygrindelwald',
 'johnnydeppedits',
 'johnnydeppart',
 'johnnydeppismylife',
 'johnnydeppsexy',
 'johnnydeppmovie',
 'johnnydeppdrawing',
 'johnnydepp_love_forever',
 'johnnydeppobsessed',
 'johnnydeppissexy',
 'johnnydeppfriends',
 'johnnydeppismine',
 'johnnydeppquetos',
 'johnnydeppdouble',
 'johnnydeppbr'

In [805]:
url4="https://api.instagram.com/v1/tags/search?q=adventure&access_token=%s" %(APP_ACCESS_TOKEN)
f = urllib.request.urlopen(url4)
myfile3 = f.read()
x3=json.loads(myfile3)

count_x3=[]
for each in x3['data']:
    count_x3.append(each['media_count'])
    
count_x3
name_x3=[]
for each in x3['data']:
    name_x3.append(each['name'])
name_x3
count_x3

act_url3="https://api.instagram.com/v1/tags/search?q=britneyspears&access_token=%s" %(APP_ACCESS_TOKEN)
act_f3 = urllib.request.urlopen(act_url3)
act_myfile3 = act_f3.read()
act_x3=json.loads(act_myfile3)


act_count3=[]
for each in act_x3['data']:
    act_count3.append(each['media_count'])
    
act_count3
act_name3=[]
for each in act_x3['data']:
    act_name3.append(each['name'])
act_name3

['britneyspears',
 'britneyspearsfan',
 'britneyspearsarmy',
 'britneyspearsqueen',
 'britneyspearsfans',
 'britneyspearscollection',
 'britneyspearsperfume',
 'britneyspearsglory',
 'britneyspearsconcert',
 'britneyspearspieceofme',
 'britneyspearsdiva',
 'britneyspearssweden',
 'britneyspearsvegas',
 'britneyspearsliveinbangkok',
 'britneyspearsmnl',
 'britneyspearsalbania',
 'britneyspearsfantasy',
 'britneyspearsfragrances',
 'britneyspearsandiggyazalea',
 'britneyspearsfanpage',
 'britneyspearsamericandream',
 'britneyspearsliveinmanila',
 'britneyspearslive',
 'britneyspearsaustralia',
 'britneyspearscollector',
 'britneyspearsthailand',
 'britneyspearsimpersonator',
 'britneyspearsmidnightfantasy',
 'britneyspearsdoll',
 'britneyspearsliveinconcert',
 'britneyspears2007',
 'britneyspearsmerch',
 'britneyspearsmoment',
 'britneyspearskuwait',
 'britneyspearscover',
 'britneyspearschile',
 'britneyspearsbkk2017',
 'britneyspearsfragrance',
 'britneyspearslasvegas',
 'britneyspears

In [806]:
url5="https://api.instagram.com/v1/tags/search?q=fiction&access_token=%s" %(APP_ACCESS_TOKEN)
f = urllib.request.urlopen(url5)
myfile4 = f.read()
x4=json.loads(myfile4)

count_x4=[]
for each in x4['data']:
    count_x4.append(each['media_count'])
    
count_x4
name_x4=[]
for each in x4['data']:
    name_x4.append(each['name'])
name_x4
count_x4

act_url4="https://api.instagram.com/v1/tags/search?q=robertdowney&access_token=%s" %(APP_ACCESS_TOKEN)
act_f4 = urllib.request.urlopen(act_url4)
act_myfile4 = act_f4.read()
act_x4=json.loads(act_myfile4)


act_count4=[]
for each in act_x4['data']:
    act_count4.append(each['media_count'])
    
act_count4
act_name4=[]
for each in act_x4['data']:
    act_name4.append(each['name'])
act_name4

['robertdowneyjr',
 'robertdowney',
 'robertdowneyjunior',
 'robertdowneyjnr',
 'robertdowneyjredit',
 'robertdowneyjrfan',
 'robertdowneyjrâ',
 'robertdowneyjr_48',
 'robertdowneyjrbigfan',
 'robertdowneyjr65',
 'robertdowneyj',
 'robertdowneyjrfans',
 'robertdowneysr',
 'robertdowneyjrpleasejointwitterin2013',
 'robertdowneyjrturkish',
 'robertdowneyjrisperfect',
 'robertdowneyjritalia',
 'robertdowneyjriloveyou',
 'robertdowneyjrdrawing',
 'robertdowneyjn',
 'robertdowneyjr_47',
 'robertdowneyjrquotes',
 'robertdowneyjrart',
 'robertdowneyjrita',
 'robertdowneyjesus',
 'robertdowneyjrquote',
 'robertdowneyjrvine',
 'robertdowneyjuniorfanpage']

In [807]:
url6="https://api.instagram.com/v1/tags/search?q=comedy&access_token=%s" %(APP_ACCESS_TOKEN)
f = urllib.request.urlopen(url6)
myfile5 = f.read()
x5=json.loads(myfile5)

count_x5=[]
for each in x5['data']:
    count_x5.append(each['media_count'])
    
count_x5
name_x5=[]
for each in x5['data']:
    name_x5.append(each['name'])
name_x5
count_x5

act_url5="https://api.instagram.com/v1/tags/search?q=vindiesel&access_token=%s" %(APP_ACCESS_TOKEN)
act_f5 = urllib.request.urlopen(act_url5)
act_myfile5 = act_f5.read()
act_x5=json.loads(act_myfile5)


act_count5=[]
for each in act_x5['data']:
    act_count5.append(each['media_count'])
    
act_count5
act_name5=[]
for each in act_x5['data']:
    act_name5.append(each['name'])
act_name5

['vindiesel',
 'vindieselfans',
 'vindiesellovers',
 'vindieselthepug',
 'vindieselfan',
 'vindieselthebrindlepug',
 'vindieselforeverfastfamily',
 'vindiesel😍',
 'vindieselinmanila',
 'vindiesellookalike',
 'vindieselstyle',
 'vindieselandpaulwalker',
 'vindiesellove',
 'vindieselvoice',
 'vindieselquotes',
 'vindieselgallery',
 'vindieselmyhero',
 'vindieselbrasileiro',
 'vindieselmovies',
 'vindieselitalia',
 'vindieselthepup',
 'vindieselthedog',
 'vindiesels',
 'vindieselquote',
 'vindieselandmichellerodriguez']

In [808]:
url7="https://api.instagram.com/v1/tags/search?q=thriller&access_token=%s" %(APP_ACCESS_TOKEN)
f = urllib.request.urlopen(url7)
myfile6 = f.read()
x6=json.loads(myfile6)

count_x6=[]
for each in x6['data']:
    count_x6.append(each['media_count'])
    
count_x6
name_x6=[]
for each in x6['data']:
    name_x6.append(each['name'])
name_x6
count_x6

act_url6="https://api.instagram.com/v1/tags/search?q=hughjackman&access_token=%s" %(APP_ACCESS_TOKEN)
act_f6 = urllib.request.urlopen(act_url6)
act_myfile6 = act_f6.read()
act_x6=json.loads(act_myfile6)


act_count6=[]
for each in act_x6['data']:
    act_count6.append(each['media_count'])
    
act_name6=[]
for each in act_x6['data']:
    act_name6.append(each['name'])
act_name6

['hughjackman',
 'hughjackmanfan',
 'hughjackmanfans',
 'hughjackmanwolverine',
 'hughjackmann',
 'hughjackmanedit',
 'hughjackmanforever',
 'hughjackmanedits',
 'hughjackmanlookalike',
 'hughjackmanlabirgece',
 'hughjackmanfanclub',
 'hughjackmanisthewolverine',
 'hughjackmanstyle',
 'hughjackmanbrasil',
 'hughjackmanlogan',
 'hughjackmanlove',
 'hughjackmansmile',
 'hughjackman_singing',
 'hughjackmanargentina',
 'hughjackmanbroadwaytooz',
 'hughjackman_persianfans',
 'hughjackmanworldbrasil',
 'hughjackmanfanpage',
 'hughjackmannobrasil',
 'hughjackmaniswolverine']

In [809]:
url8="https://api.instagram.com/v1/tags/search?q=animation&access_token=%s" %(APP_ACCESS_TOKEN)
f = urllib.request.urlopen(url8)
myfile7 = f.read()
x7=json.loads(myfile7)

count_x7=[]
for each in x7['data']:
    count_x7.append(each['media_count'])
    
name_x7=[]
for each in x7['data']:
    name_x7.append(each['name'])


act_url7="https://api.instagram.com/v1/tags/search?q=halleberry&access_token=%s" %(APP_ACCESS_TOKEN)
act_f7 = urllib.request.urlopen(act_url7)
act_myfile7 = act_f7.read()
act_x7=json.loads(act_myfile7)


act_count7=[]
for each in act_x7['data']:
    act_count7.append(each['media_count'])
    
act_count7
act_name7=[]
for each in act_x7['data']:
    act_name7.append(each['name'])
act_name7

['halleberry',
 'halleberryhair',
 'halleberrycut',
 'halleberryorhallelujah',
 'halleberryhallelujah',
 'halleberryhalleberry',
 'halleberryfeet',
 'halleberrystyle',
 'halleberrythedog',
 'halleberry❤️',
 'halleberryratchettime',
 'halleberrywho',
 'halleberryperfume',
 'halleberryhaircut',
 'halleberrylookalike',
 'halleberrystorm',
 'halleberryice',
 'halleberrys',
 'halleberrypup',
 'halleberryeliquid',
 'halleberryinspired',
 'halleberrycatwoman',
 'halleberrythecat',
 'halleberryejuice',
 'halleberrylook',
 'halleberryfans',
 'halleberrycf',
 'halleberrymoment',
 'halleberryfan',
 'halleberrywannabe',
 'halleberrymaltese',
 'halleberrystatus',
 'hallèberry',
 'halleberrycloser',
 'halleberrylujah',
 'halleberryfine',
 'halleberrycorgi',
 'halleberryvibes',
 'halleberryvoice',
 'halleberrygroovers',
 'halleberrymovie',
 'halleberryfam',
 'halleberryshort',
 'halleberrylewjah',
 'halleberrywildessence',
 'halleberryedit',
 'halleberrywig',
 'halleberrycat',
 'halleberryswag',
 'ha

In [810]:
url9="https://api.instagram.com/v1/tags/search?q=drama&access_token=%s" %(APP_ACCESS_TOKEN)
f = urllib.request.urlopen(url9)
myfile8 = f.read()
x8=json.loads(myfile8)

count_x8=[]
for each in x8['data']:
    count_x8.append(each['media_count'])
    
count_x8
name_x8=[]
for each in x8['data']:
    name_x8.append(each['name'])
name_x8
count_x8
len(count_x)

act_url8="https://api.instagram.com/v1/tags/search?q=jackiechan&access_token=%s" %(APP_ACCESS_TOKEN)
act_f8= urllib.request.urlopen(act_url8)
act_myfile8 = act_f8.read()
act_x8=json.loads(act_myfile8)


act_count8=[]
for each in act_x8['data']:
    act_count8.append(each['media_count'])
    
act_count8
act_name8=[]
for each in act_x8['data']:
    act_name8.append(each['name'])
act_name8

['jackiechan',
 'jackiechanproducoes',
 'jackiechanstyle',
 'jackiechanproduções',
 'jackiechanadventures',
 'jackiechanfans',
 'jackiechandcracing',
 'jackiechanmovie',
 'jackiechanstuntteam',
 'jackiechang',
 'jackiechanfan',
 'jackiechanstar',
 'jackiechanmeme',
 'jackiechanracing',
 'jackiechanjointculture',
 'jackiechanmovies',
 'jackiechanface',
 'jackiechanstuntmaster',
 'jackiechanfansclub',
 'jackiechangroupkorea',
 'jackiechanningtatum',
 'jackiechans',
 'jackiechantakeover',
 'jackiechantakhtay',
 'jackiechanandchristucker',
 'jackiechan_concert',
 'jackiechantheram',
 'jackiechanmoney',
 'jackiechansuperpug',
 'jackiechance',
 'jackiechangroup',
 'jackiechanlookalike']

In [811]:
genre_list =['action','horror','romance','adventure','fiction','comedy','thriller' ,'animation','drama']
import unicodedata

genre_tags=[name_x,name_x1,name_x2,name_x3,name_x4,name_x5,name_x6,name_x7,name_x8]

genre_count=[count_x,count_x1,count_x2,count_x3,count_x4,count_x5,count_x6,count_x7,count_x8]

actor_list =['Dwayne Johnson','Brad Pitt','Johnny Depp','Britney Spears','Robert Downey JR.','Vin Diesel','Hugh Jackman' ,'Halle Berry','Jackie Chan']

actor_tags=[act_name,act_name1,act_name2,act_name3,act_name4,act_name5,act_name6,act_name7,act_name8]

actor_count=[act_count,act_count1,act_count2,act_count3,act_count4,act_count5,act_count6,act_count7,act_count8]


In [812]:
genre_insta_tags = []
genre_insta_counts = []
genre_insta_name = []

actors_insta_tags = []
actors_insta_counts =[]
actors_insta_name =[]

for i,x in enumerate(genre_tags):
    for y in x:
        genre_insta_tags.append(y)
        genre_insta_name.append(genre_list[i])
        
for x in genre_count:
    for y in x:
        genre_insta_counts.append(y)
        
for i,x in enumerate(actor_tags):
    for y in x:
        actors_insta_tags.append(y)
        actors_insta_name.append(actor_list[i])

for x in actor_count:
    for y in x:
        actors_insta_counts.append(y)

In [813]:
genre_insta_df = pd.DataFrame(genre_insta_name, columns = ['Genre'])
genre_insta_df['Tags'] = genre_insta_tags
genre_insta_df['Counts'] = genre_insta_counts

actors_insta_df = pd.DataFrame(actors_insta_name, columns=['Actor'])
actors_insta_df['Tags'] = actors_insta_tags
actors_insta_df['Counts'] = actors_insta_counts

# Learning SQL 

#### What is SQL?

SQL (Structured Query Language) is the primary language responsible for managing data and data structures contained within a relational database management system (RDBMS). Put simply, SQL is the language you use to interact with a database. There are four basic operations that SQL can perform: INSERTs, SELECTs, UPDATEs, and DELETEs (these are sometimes referred to as CRUD operations - create, read, update, delete).

#### What are some database concepts?

A database, in simplest terms, is an organized collection of data. A database, as we know is comprised of many tables, and a table stores rows of data in a structured format defined by the table's columns. This represents the basic hierarchy of a database. When writing SQL queries, you are interacting with rows of data stored in tables contained within a database.

#### Why SQL?

SQL queries can be easily saved and re-used at any point in time. SQL code can also be edited with comments, so you can include clear descriptions directly in your query. When you work with Excel, you might be stuck doing a long multi-step process. First, you might have to export data from some standard report, then sort it, then add headers, then filter out certain values etc.

With SQL code, you simply write the code once, save it, re-open it, and re-run it if you ever need to produce a report twice. You do not have to manually produce reports that could easily be automated. All the questions you ask about your data on a regular basis can be answered using SQL is what we are trying to learn in this project. Once you identify the database that stores the data you seek to explore, SQL empowers you to answer your own questions.

## SQL SCHEMA

<img src = 'SQL_Final_Schema.png'>

## SQL Database Design 

As shown in the schema above, we have seperate tables for Movies, Production companies, Shoot location, Genre, Cast, Tags and Twitter Actor Data, Twitter Public data, Facebook production comapnies, Facebook Public data, Instagram actors, Instagram Genres.<br> 

Initially, there were only two tables (Cast and Movies) that contained all the movie information. However, the tables were expanded to Cast, Movies, Production companies, Shoot Venue and Genre in order to make each table atomic and to bring it to first normal form. <br>
<br>
The Tags table contains information on what is being said about a particular actor in Twitter, what are the posts on the 'production comapnies' page on FB and what tags related to the genres are being used in Instagram. 

This was achieved by scraping public data which had the actors name in them, and then storing the hashtags present in those posts. For example, if we search for 'Tom Cruise' on twitter, thousands of posts pop up which either have Tom Cruise in the status or a hashtag with the word 'Tom Cruise' in them. The hashtags from these posts are databased for all the actors of all the movies we have in our Movies Database and stored in the Tags table. <br>
<br>
The Twitter_actor_data table contains information on the posts, retweets, followers etc of each actor. It was created by scraping each actors twitter timeline individually and databasing all the information related to each post. <br>
<br>

Similarly,for facebook, we extract the tags, no. of likes, comments and shares from each post from all the 6 'production company' pages. Store all of this data in a dataframe and then do further analysis. The facebook_public_data consists of the posts which contain similar hashtags.

All the tables shown in the schema are in their first normal form with six tables (Twitter Actor Data, Twitter Public data, Facebook production comapnies, Facebook Public data, Instagram actors, Instagram Genres) being tables created by scraping Twitter, Facebook and Instagram.

# Importing library for SQLite

In [494]:
import sqlite3
conn = sqlite3.connect('TMDB_Database')

# Converting the dataframes into SQL Tables 

In [495]:
movies_df.to_sql('movies',con=conn,index=False, if_exists='replace')
countries_df.to_sql('shoot_venue', con=conn, index=False, if_exists='replace')

In [496]:
prod_companies_df.to_sql('production_companies', con=conn, index=False, if_exists='replace')
cast_df.to_sql('cast', con=conn, index=False, if_exists='replace')
genre_df.to_sql('genre', con=conn, index=False, if_exists='replace')

In [559]:
tags_df.to_sql('tags', con=conn, index=False, if_exists='replace')
twitter_data_df.to_sql('Twitter_Data', con=conn, if_exists='replace', index=False)
public_tweets_df.to_sql('Public_Twitter_Data',con=conn, if_exists='replace',index=False)
prod_comp_fb_df.to_sql('Facebook_data', con=conn, if_exists='replace', index=False)
facebook_public_data.to_sql('Public_Facebook_Data',con = conn, if_exists='replace', index=False)

In [560]:
def run_query(query):
    return pd.read_sql(query,conn)

# PROJECT 1 - SQL 

## Answers Using Twitter

### (Answers are with respect to actors)

## Q1 What are people saying about me (Anupam Kher)? 

In [561]:
run_query('SELECT name,tags FROM tags WHERE name = "Anupam Kher"') # Select the name and tags associated with Anupam Kher

,name,tags
0,Anupam Kher,AnupamKher
1,Anupam Kher,BiggestStar
2,Anupam Kher,HumanBeing
3,Anupam Kher,Sridevi


We see the associated tags with the famous indian actor 'Anupam Kher' as tweeted by people on twitter.

## Q2 How viral are my (Anupam Khers) posts?

In [565]:
run_query('SELECT name,AVG(Favourites), AVG(Retweets), COUNT(*) AS No_of_posts FROM twitter_data WHERE Name = "Anupam Kher"')
#Select the average number of favourites, retweets and the total number of posts of Anupam Kher

,Name,AVG(Favourites),AVG(Retweets),No_of_posts
0,Anupam Kher,1885.77,348.9,100


We see that Anupam Kher gets an average of 1900 Favourites and an average of 350 Retweets per post for each of the 100 posts  we have databased.

## Q3 How much influence do my (Dwayne Johnson) posts have? 

In [566]:
run_query('SELECT name,Followers, AVG(Favourites), AVG(Retweets) FROM twitter_data where Name = "Dwayne Johnson"')
#Select dwayne johnsons no. of followers, average no of likes and retweets

,Name,Followers,AVG(Favourites),AVG(Retweets)
0,Dwayne Johnson,12827195,2704.05,293.41


We see that Dwayne Johnson A.K.A The Rock! has 12 million followers and 2700 people like his posts on average showing he is highly influencial!

## Q4 What posts are like mine (Janet Jackson)?

First, lets take a look at Janet Jacksons posts

In [572]:
run_query('SELECT name,post, hashtags from twitter_data WHERE name = "Janet Jackson" LIMIT 3')

,Name,Post,Hashtags
0,Janet Jackson,Attention... it’s time to #DANCEWITHJANET! Sho...,DANCEWITHJANET
1,Janet Jackson,"Dr. #MayaAngelou, the original #PhenomenalWoma...",MayaAngelou
2,Janet Jackson,So happy we met and so proud of all that you’r...,N/A


We see that Janet Jackson has posted about the late American poet, singer, memoirist, and civil rights activist Maya Angelou. We now look at public posts on twitter that are similar

In [575]:
run_query('SELECT post FROM public_twitter_data WHERE post LIKE "%Maya%Angelou%"') 
#Select posts from public tweets that have the words "Maya Angelou" in them

,post
0,"'If you're always trying to be 'normal', you w..."
1,"“I had a lot of clouds, but I have had so many..."
2,RT @ABAonline: 'If you're always trying to be ...
3,Our final post for National Poetry Month is Ma...
4,"#Kanye ""When someone shows you who they are, b..."
5,#MayaAngelou #stillirise #positivity #strength...
6,THIS here is why #ILoveDance\n\nhttps://t.co/w...
7,Maya's Mind is a towering and majestic sculptu...
8,RT @IMQuotes_Videos: Make it your goal to THRI...
9,R E V O L U T I O N // Maya Angelou once said:...


## Q5 What users post like me? (Janet Jackson)

Let us now look at the names of users who have similar posts to that of Janet Jackson

In [582]:
run_query('SELECT name, post FROM public_twitter_data WHERE post LIKE "%Maya%Angelou%" OR post LIKE "DANCE" LIMIT 5')
#Select all the public tweets that have the words "Maya Angelou" or "Dance" in them

,name,post
0,AntiBullyingAlliance,"'If you're always trying to be 'normal', you w..."
1,Maya Angelou,"“I had a lot of clouds, but I have had so many..."
2,The JoP ♥ ✌️,RT @ABAonline: 'If you're always trying to be ...
3,OmahaChildren'sChoir,Our final post for National Poetry Month is Ma...
4,Husky Haddish,"#Kanye ""When someone shows you who they are, b..."


It appears that Maya Angelou (duh), AntiBullyingAlliance, OmahaChildrensChoir, Husky Haddish are some of the users/pages on twitter that post like Janet Jackson

## Q6 Who should I (Janet Jackson) be following?

In [586]:
run_query('SELECT name, followers FROM public_twitter_data WHERE post LIKE "%Maya%Angelou%"')

,name,followers
0,AntiBullyingAlliance,21592
1,Maya Angelou,614908
2,The JoP ♥ ✌️,2307
3,OmahaChildren'sChoir,492
4,Husky Haddish,2238
5,Seth Pedone,24
6,Douglas Connors,1632
7,Marita Golden,1068
8,Mz_Necie_♏🇯🇲,217
9,Conscious & Chic™,1534


Janet Jackson should be following AntiBullyingAlliance, Maya Angelou (surprise!) and Husky Hadish as they seem to have a high number of followers (21k, 6 million and 2k respectively) and post similar content like Janet Jackson

## Q7 What topics are trending in my domain?

To answer this question, we look at the most used hashtags in April 2018 by various actors

In [600]:
run_query('SELECT count(*),hashtags,posted_time FROM twitter_data GROUP BY hashtags HAVING posted_time LIKE "%Apr%2018%" ORDER BY Count(*) DESC LIMIT 5')
# Select count of the tweets containing repeated hashtags on twitter in April 2018

,count(*),Hashtags,Posted_time
0,13,TheAccidentalPrimeMinister,Thu Apr 05 12:32:20 +0000 2018
1,4,RAMPAGE,Wed Apr 11 20:04:52 +0000 2018
2,4,Rampage,Wed Apr 11 22:55:35 +0000 2018
3,3,ManmohanSingh,Thu Apr 05 11:38:09 +0000 2018
4,2,AnupamKher,Wed Apr 11 22:28:12 +0000 2018


It looks like the movies "The Accidental Prime Minister" starring Anupam Kher and Dwayne Johnsons new movie "RAMPAGE" is trending according to their repeated occurrences in the tags

## Q8 What keywords/hashtags should I add to my post?

We can add the keywords/hashtags that frequently occur as those are more likely to get viewed and retweeted by others

In [606]:
run_query('select count(*), hashtags from twitter_data group by hashtags having hashtags != "N/A"order by count(*) desc LIMIT 5')

,count(*),Hashtags
0,15,SOTW
1,13,TheAccidentalPrimeMinister
2,5,JanFam
3,4,RAMPAGE
4,4,Rampage


We can add the above tags in order to get more views/likes/comments on our posts

## Q9 Should I (Janet Jackson) follow somebody back?

We should follow those people that post similar content like us and are highly influencial back

In [685]:
run_query('SELECT name, followers FROM public_twitter_data WHERE post LIKE "%Maya%Angelou%" ORDER BY followers DESC LIMIT 5')
# Select the people in descending order of followers and who posted content relating to Maya Angelou

,name,followers
0,Maya Angelou,614908
1,AntiBullyingAlliance,21592
2,The JoP ♥ ✌️,2307
3,Husky Haddish,2238
4,Douglas Connors,1632


We should follow the above people back asa they have high number of followers

## Q10 What is the best time to post? 

In [617]:
run_query('SELECT posted_time,favourites,retweets from twitter_data GROUP BY Posted_time ORDER BY Favourites DESC LIMIT 7')
#Select posts with high number of favourites and retweets grouping by posted time

,Posted_time,Favourites,Retweets
0,Wed Feb 21 00:36:24 +0000 2018,618861,139309
1,Sat Apr 15 01:17:29 +0000 2017,112490,29431
2,Tue Apr 03 20:30:33 +0000 2018,29820,11037
3,Wed Oct 11 17:24:13 +0000 2017,26536,8450
4,Sat Nov 25 17:29:20 +0000 2017,23619,3737
5,Fri Apr 13 17:55:03 +0000 2018,22085,3524
6,Thu Mar 08 18:07:16 +0000 2018,20922,3982


We see that on average, posts between 17:00:00 (5pm) and 20:00:00 (8pm) have higher number of favourites/retweets

## Q11 Should I add a picture or url to my post?

Let us check whether posts with url/pictures in them are popular or not 

In [622]:
run_query('SELECT name,post,favourites,retweets FROM twitter_data WHERE post LIKE "%https%" ORDER BY Favourites DESC LIMIT 10')
# Select the posts with https://.. in them to see whether they have a high number of likes

,Name,Post,Favourites,Retweets
0,Janet Jackson,My baby and me after nap time. https://t.co/5s...,112490,29431
1,Janet Jackson,The Beat Goes On. https://t.co/VXDXs4Y5pM,26536,8450
2,Janet Jackson,Can you spot me? 😉 https://t.co/JN56dAEd4I,23619,3737
3,Dwayne Johnson,Wow! Very cool RAMPAGE news! Not pointing to t...,22085,3524
4,Janet Jackson,"Our time has come to rejoice, a new agenda’s d...",20922,3982
5,Janet Jackson,A special message from Janet for her fans. htt...,20901,7593
6,Dwayne Johnson,🤣 Let me assist this very important global pol...,19778,3118
7,Janet Jackson,Another 1 from the archives. https://t.co/T7l0...,17476,3650
8,Janet Jackson,Hey u guys. I’m so excited for the State of th...,15248,3573
9,Janet Jackson,"Last night in LA, generations of my former dan...",15096,4716


We see that posts with url/pictures in them do have a high number of likes and retweets and hence it is recommended to add a url/picture to your post

## Q12 Whats my reach ? (Danny Glover)

We can check Danny Glovers reach by looking at the response he gets for his posts

In [631]:
run_query('SELECT name,post,followers,(favourites), (retweets) FROM twitter_data WHERE name = "Danny Glover" LIMIT 10')

,Name,Post,Followers,Favourites,Retweets
0,Danny Glover,"RT @BWOPATILE: Actor, Producer, Humanitarian @...",1466022,10,4
1,Danny Glover,New Poll Shows Pamela Price Leading Nancy O’Ma...,1466022,27,7
2,Danny Glover,"Sundance Winner ‘Hale County This Morning, Thi...",1466022,27,9
3,Danny Glover,RT @TheRealNews: During a Q&amp;A with actor @...,1466022,98,64
4,Danny Glover,RT @MarkWeisbrot: Great statement from Ro Khan...,1466022,15,9
5,Danny Glover,RT @TheSandersInst: Why do we vote on a Tuesda...,1466022,762,432
6,Danny Glover,RT @RepRoKhanna: I am very concerned that the ...,1466022,1076,402
7,Danny Glover,"RT @I_AM_2018: ""I hope and pray that this gath...",1466022,44,16
8,Danny Glover,RT @MajorCBS: A discussion on political activi...,1466022,16,14
9,Danny Glover,"RT @AFSCME: ""We gotta do what we do best - org...",1466022,41,26


We see that Danny Glover has only very few favourites/retweets for each of his posts but has 14 million followers which shows that either he is not very influential or his reach is below average

# PROJECT 1 - SQL

## Answers Using Facebook

### (Answers are with respect to Production Companies)

## Q1 What are people saying about me (Disney)? 

In [638]:
run_query('SELECT name,post FROM public_facebook_data WHERE post LIKE "%Disney%" ')
#Query public facebook posts with the word 'Disney' in them 

,name,post
0,Disney,🚨 50 days. 🚨 #disney #Incredibles2 in theatres...
1,Helen Parr(Elastic Girl),See Elastigirl and The Incredibles in Disney H...
2,Rtr Maitri Vasa,#Incredibles #November #disneystudios
3,Anjaneya Shetty,GameSpots PostThe Incredibles are almost back!...
4,Keerthana Subramanian,OMGOMGOMGOMG THEY ARE BACKKKKKK Madhuvanthi ht...
5,Ankit Dhame,FINALLY Disneys PostThe Incredibles 2 trailer ...
6,Tanner Tucker,This new trailer is great 😂😱👏🏼 so pumped for t...
7,The Incredibles,Congratulations to The Royal Family on their I...
8,Disney PIxar,🚨 50 days! 🚨 See #Incredibles2 in theatres Jun...


## Q2 How viral are my (Disney's) posts?

In [646]:
run_query('SELECT production_company,AVG(likes), AVG(shares), AVG(comments) FROM facebook_data WHERE production_company = "Disney" LIMIT 10')

,Production_company,AVG(likes),AVG(shares),AVG(comments)
0,Disney,3942.06,1352.82,444.33


We see that Disney gets on average 4000 likes, 1300 shares and 500 comments per post

## Q3 How much influence do my (Universal Studios) posts have? 

In [649]:
run_query('SELECT production_company,Page_likes, AVG(Likes), AVG(Shares), AVG(Comments) FROM facebook_data WHERE production_company = "Universal Studios"')

,Production_company,Page_likes,AVG(Likes),AVG(Shares),AVG(Comments)
0,Universal Studios,8253534,1518.32,626.05,184.63


Universal studios has 8.2 million followers with an average of 1500 likes per post establishing their highly influencial digital presence

## Q4 What posts are like mine ? (Disney)

First lets take a look at a few of Disneys posts

In [655]:
run_query('SELECT * FROM facebook_data WHERE production_company = "Disney" LIMIT 4')

,Production_company,Page_likes,Posts,Hashtags,Likes,Shares,Comments,Created_time
0,Disney,51176895,"Don’t freeze. Chadwick Boseman, Danai Gurira, ...",ForcesOfDestiny,601,89,16,2018-04-26T01:00:01+0000
1,Disney,51176895,Delicious news! The Disney Eats Collection is ...,ad,862,165,89,2018-04-25T23:00:03+0000
2,Disney,51176895,Entertainment Weekly has an exclusive new look...,Incredibles2,2329,406,157,2018-04-25T21:16:28+0000
3,Disney,51176895,Think of the happiest thing! Peter Pan flies o...,Incredibles2,1228,286,175,2018-04-25T17:00:01+0000


We see Disney promoting their upcoming movie Incredibles 2 quite a lot. Lets see what the public has to say about the Incredibles

In [660]:
run_query('SELECT post FROM public_facebook_data WHERE post LIKE "%Incredibles%"')

,post
0,🚨 50 days. 🚨 #disney #Incredibles2 in theatres...
1,See Elastigirl and The Incredibles in Disney H...
2,From Avengers: Infinity War to The Incredibles...
3,#Incredibles #November #disneystudios
4,GameSpots PostThe Incredibles are almost back!...
5,OMGOMGOMGOMG THEY ARE BACKKKKKK Madhuvanthi ht...
6,FINALLY Disneys PostThe Incredibles 2 trailer ...
7,Congratulations to The Royal Family on their I...
8,#incredibles Even though Incredibles is probab...
9,🚨 50 days! 🚨 See #Incredibles2 in theatres Jun...


## Q5 What users post like me? (Disney)

Lets look at the names of the users who posted content about the Incredibles

In [665]:
run_query('SELECT name FROM public_facebook_data WHERE post LIKE "%Incredibles%"')

,name
0,Disney
1,Helen Parr(Elastic Girl)
2,IGN
3,Rtr Maitri Vasa
4,Anjaneya Shetty
5,Keerthana Subramanian
6,Ankit Dhame
7,The Incredibles
8,John Livingston
9,Disney PIxar


## Q6 Who should I (Disney) be following ? 

We should follow people who post similar content like us and are very influential. We can measure this by ordering users posting similar content by their amount of followers.

In [672]:
run_query('SELECT name,[followers/page_likes] FROM public_facebook_data WHERE post LIKE "%Incredibles%" AND name != "Disney"  ORDER BY [followers/page_likes] DESC LIMIT 5')

,name,followers/page_likes
0,Disney PIxar,14020065
1,The Incredibles,7322483
2,IGN,5173840
3,Helen Parr(Elastic Girl),2036311
4,Rtr Maitri Vasa,2316


Quite obviously, we see that Disney should be following Disney Pixar, The Incredibles page, IGN etc

## Q7 What topics are trending in my domain ? 

We check the most frequently used hashtags (by count) in April 2018

In [679]:
run_query('SELECT Hashtags, created_time, COUNT(*) FROM facebook_data GROUP BY Hashtags HAVING created_time LIKE "%2018%04%" AND hashtags != "N/A" ORDER BY COUNT(*) DESC LIMIT 5')

,Hashtags,Created_time,COUNT(*)
0,Incredibles2,2018-04-17T16:54:01+0000,16
1,SuicideSquadHTP,2018-04-03T15:46:43+0000,9
2,HappyBirthday,2018-04-06T14:00:01+0000,6
3,BatmanNinja,2018-04-19T18:00:06+0000,5
4,AQuietPlace,2018-04-05T19:13:10+0000,4


We see the movies Incredibles2 (used in tags 16 times), SuicideSquad, HappyBirthday, batmanNinja and A Quiet Place (playing now in cinemas near you) are all trending!

## Q8 What keywords/hashtags should i add to my posts ?

We should add those keywords/hashtags that are the most popular and are likely to get more views/comments/likes

In [683]:
run_query('SELECT Hashtags, COUNT(*) FROM facebook_data GROUP BY Hashtags HAVING hashtags!="N/A" ORDER BY COUNT(*) DESC LIMIT 7')

,Hashtags,COUNT(*)
0,Incredibles2,16
1,DirtMovie,9
2,SuicideSquadHTP,9
3,JusticeLeague,8
4,LegoFlash,8
5,StayQuiet,7
6,FantasticBeastsVR,6


We can add any of the hashtags above in order to get more likes/views

## Q9 Should I (Disney) follow somebody back ?

We should follow those people that post similar content like us and who are influential, back

In [686]:
run_query('SELECT name,[followers/page_likes] FROM public_facebook_data WHERE post LIKE "%Incredibles%" AND name != "Disney"  ORDER BY [followers/page_likes] DESC LIMIT 5')

,name,followers/page_likes
0,Disney PIxar,14020065
1,The Incredibles,7322483
2,IGN,5173840
3,Helen Parr(Elastic Girl),2036311
4,Rtr Maitri Vasa,2316


We should follow the above people/pages back

## Q10 What is the best time to post ? 

Let us order the posts in descending order of no. of likes in order to see the time at which posts get the maximum no of likes shares and comments

In [696]:
run_query('SELECT posts,Likes,comments, shares,created_time FROM facebook_data ORDER BY likes DESC LIMIT 7')

,Posts,Likes,Comments,Shares,Created_time
0,Watch the new Avengers: Infinity War trailer. ...,240801,37833,242639,2018-03-16T13:03:56+0000
1,"“There was an idea, to bring together a group ...",200156,8137,80587,2018-04-04T19:00:00+0000
2,Check out these new images from Avengers: #Inf...,94911,8206,61361,2018-03-12T20:01:12+0000
3,10 years. Over 18 movies. A special thank you ...,88459,3373,41672,2018-04-10T16:00:00+0000
4,An entire universe. Once and for all. Avengers...,83167,3915,44458,2018-03-26T19:00:19+0000
5,An entire Universe. Once and for all. Check ou...,61977,1012,13377,2018-04-24T20:04:06+0000
6,Where will you be when it all ends? See Avenge...,55145,5234,32098,2018-04-07T16:00:00+0000


We see that the posts with the highest number of Likes /comments/shares are posted between 4pm and 8pm

## Q11 Should i add a picture/url to my post ?

Let us check whether posts with pictures/urls in them are popular or not

In [700]:
run_query('SELECT posts, likes, shares,comments FROM facebook_data  WHERE Posts LIKE "%https%"')
#Select posts with https in them to see their likes,shares,comments

,Posts,Likes,Shares,Comments
0,Never stop believing - because it's never too ...,549,89,38
1,"Hey Boondock Saints fans, you are invited to j...",0,0,0
2,Learn more about The Road to Infinity War and ...,13871,936,130
3,Ant-Man and the Wasp is one of five covers fea...,11141,539,93
4,Get a first look at Johnny Knoxville's new mov...,46,4,6
5,The Joker and Harley Quinn are double trouble ...,231,21,13
6,Batman never backs down from a challenge! See ...,448,89,49
7,Bat Family to the rescue! Own #BatmanNinja on ...,684,171,64
8,With her eco-terrorism unleashed over Feudal J...,227,19,9
9,"Gotham’s criminals have gone wild in Japan, wi...",1808,592,110


We see that most posts have high no. of likes and a few dont. Hence, it is recommended to add pictures/urls to the post

## Q12 Whats my (Paramount Studios) reach ? 

We can check a studios reach by checking their followers and the average no. of likes, shares and comments they get on each post

In [704]:
run_query('SELECT production_company,Page_likes, AVG(likes), AVG(shares), AVG(comments) FROM facebook_data WHERE production_company = "Paramount"')

,Production_company,Page_likes,AVG(likes),AVG(shares),AVG(comments)
0,Paramount,11459910,1180.38,1473.47,446.91


We see that Paramount has 11 million followers with an average of 1200 likes, 1500 shares and 250 comments/post

# SQL vs NoSQL

In the SQL part we have explained our design considerations and how to analyze the data using SQL queries. Here, we discuss how we can get the answers to those same questions using pymongo (NO SQL)and why non-relationl NoSQL (MongoDb/PyMongo) is better than relational SQL databases.

In relational databases you,

    i.store related data in tables
    ii.require a schema which defines tables prior to use
    iii. encourage normalization to reduce data redundancy
    iv. support table JOINs to retrieve related data from multiple tables in a single command
    v. implement data integrity rules
    vi. provide transactions to guarantee two or more updates succeed or fail as an atomic unit
    vii. can be scaled (with some effort)
    viii. use a powerful declarative language for querying
    ix. offer plenty of support, expertise and tools.
    
Whereas in non-relational databases:

    i. store related data in JSON-like, name-value documents
    ii. can store data without specifying a schema
    iii. must usually be denormalized so information about an item is contained in a single document
    iv. do not require JOINs (presuming denormalized documents are used)
    v. permit any data to be saved anywhere at anytime without verification
    vi. guarantee updates to a single document — but not multiple documents
    vii. provide excellent performance and scalability
    viii. use JSON data objects for querying
    ix. are a newer, exciting technology.

#### SQL is digital 

SQL databases are ideal for projects where requirements can be determined and robust data integrity is essential. It works best for clearly defined, discrete items with exact specifications. Typical use cases are online stores and banking systems.

#### NoSQL is analog. 

NoSQL databases are ideal for unrelated, indeterminate or evolving data requirements where speed and scalability are more important. In simpler terms, it works best for organic data with fluid requirements. Typical use cases are social networks, customer management and web analytics systems.

# Why No SQL?

Relational databases enforces 'ACID'. So, you will have schema based transaction oriented database. It's proven and suitable for 99% of the real world applications. You can practically do anything with relational databases.

But, there are limitations on speed and scaling when it comes to massive high availability database. For example, Google and Amazon have terabytes of data stored in big data centers. Querying and inserting is not performant in these scenarios because of the blocking/schema/transaction nature of the RDBMs. That's the reason they have implemented their own databases (actually, key-value stores) for massive performance gain and scalability.

NoSQL solutions are usually meant to solve a problem that relational databases are either not well suited for, too expensive to use (like Oracle) or require you to implement something that breaks the relational nature of your database. Advantages are usually specific to usage, but unless you have some sort of problem modeling your data in a RDBMS. NoSQL allows you to persist your data as documents (or graphs) which may map much more closely to your data model.

You can use a NoSQL db when:

    i. The requirement is for a 99.999% availability on a high traffic site.
    ii. Your data makes no sense in SQL, you find yourself doing multiple JOIN queries for accessing some piece of information.
    iii. You are breaking the relational model, you have denormalized data and you generate external indexes to search that data.
    iv. Your domain objects do not fit well in a relaitional database schema


### Types of NoSQL databases-

There are 4 basic types of NoSQL databases:

    1. Key-Value Store – It has a Big Hash Table of keys & values {Example- Riak, Amazon S3 (Dynamo)}
    2. Document-based Store- It stores documents made up of tagged elements. {Example- CouchDB}
    3. Column-based Store- Each storage block contains data from only one column, {Example- HBase, Cassandra}
    4. Graph-based-A network database that uses edges and nodes to represent and store data. {Example- Neo4J}

Another example of No SQL db is MongoDB. It is an open source database that uses a document-oriented data model. MongoDB is one of several database types to arise in the mid-2000s under the NoSQL banner. Instead of using tables and rows as in relational databases, MongoDB is built on an architecture of collections and documents.



# Converting to NoSQL

Separate tables for production companies, production countries and genres are all merged into one under the movies collection. We will now deal with only two collections: Movies and Actors.<br>
<br>
Each document in the Movies collection would refer to a single movie and contain all the information pertaining to that movie like its title, description, popularity, production companies, shoot locations etc which had previously existed in different tables. It is now much more convenient to access all the information about a particular movie. <br>
<br>
Each document in the actors collection would contain all the twitter information on a particular actor including his tweets, retweets, followers, hashtags etc <br>
<br> 
The code to create the Movies collection is given below. It contains information from the Movies, Production companies, Shoot venues, Genres all merged into a list of dictionaries with each dictionary containing information on a movie. <br>

On similar lines, we have converted the FB and instagram tables as well.

In [55]:
df_raw = pd.read_csv('tmdb_5000_movies.csv')
genre_dict = df_raw['genres']                         #We store genre, production companies and shoot venue information
prod_companies_dict = df_raw['production_companies']  # in variables in order to feed them into our dictionary
countries_dict = df_raw['production_countries']
movies_nosql_dict = []
for index, row in movies_df.iterrows(): #We iterate through each row in the movies dataframe and store the information in 
    movie_dict = {}                     # a dictionary.
    movie_dict['id'] = row['id']
    movie_dict['title'] = row['title']
    movie_dict['budget'] = row['budget']
    movie_dict['revenue'] = row['revenue']
    movie_dict['original_language'] = row['original_language']      #Storing different attributes into the dictionary
    movie_dict['tagline'] = row['tagline']
    movie_dict['overview'] = row['overview']
    movie_dict['release_date'] = row['release_date']
    movie_dict['runtime'] = row['runtime']
    movie_dict['popularity'] = row['popularity']
    movie_dict['vote_average'] = row['vote_average']
    movie_dict['vote_count'] = row['vote_count']
    movie_dict['production_companies'] = prod_companies_dict[index]
    movie_dict['production_countries'] = countries_dict[index]
    movie_dict['genres'] = genre_dict[index]
    movies_nosql_dict.append(movie_dict)
movies_nosql_dict[:10]                      #Finally, we get a list of dictionaries 

[{'budget': 237000000,
  'genres': '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]',
  'id': 19995,
  'original_language': 'en',
  'overview': 'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.',
  'popularity': 150.437577,
  'production_companies': '[{"name": "Ingenious Film Partners", "id": 289}, {"name": "Twentieth Century Fox Film Corporation", "id": 306}, {"name": "Dune Entertainment", "id": 444}, {"name": "Lightstorm Entertainment", "id": 574}]',
  'production_countries': '[{"iso_3166_1": "US", "name": "United States of America"}, {"iso_3166_1": "GB", "name": "United Kingdom"}]',
  'release_date': '10-12-2009',
  'revenue': 2787965087,
  'runtime': 162.0,
  'tagline': 'Enter the World of Pandora.',
  'title': 'Avatar',
  'vote_average': 7.2,
  'vote_count': 11800},
 {'b

### Converting twitter data to its NoSQL equivalent

The code to create the Actors Collection is given below:

In [56]:
tags_dict_list=[]
tags_dict=dict()
for index, row in tags_df.iterrows():  #First we create a dictionary with the actor name as key and his tags as values
    if row['name'] in tags_dict.keys():
        tags_dict[row['name']]['tags'].append(row['tags'])
    else:
        tags_dict[row['name']] = {'count':row['count'], 'tags': [row['tags']]}

actors_dict = []
for name in tags_dict.keys():         #We then create a dictionary in the format as shown below with name,tags and count
    d = tags_dict[name]               # as the keys and its corresponding values
    d['name'] = name
    actors_dict.append(d)

In [62]:
posts_ak = list(df_1['Post']) #Adding Anupam Khers Twitter Data to Actors Dictionary
hashtags_ak = list(df_1['Hashtags'])
posttime_ak = list(df_1['Posted_time'])
favs_ak = list(df_1['Favourites'])
retweets_ak = list(df_1['Retweets'])
User_id_ak = 76294950
followers_ak = 12330588
for each in actors_dict:
    if each['name'] == 'Anupam Kher':
        each['User_id'] = User_id_ak
        each['Posts'] = posts_ak
        each['Hashtags'] = hashtags_ak
        each['Posted_Time'] = posttime_ak
        each['Favourites'] = favs_ak
        each['Retweets'] = retweets_ak
        each['Followers'] = followers_ak

In [317]:
posts_dj = list(df_2['Post']) #Adding Dwayne Johnsons Twitter Data to Actors Dictionary
hashtags_dj = list(df_2['Hashtags'])
posttime_dj = list(df_2['Posted_time'])
favs_dj = list(df_2['Favourites'])
retweets_dj = list(df_2['Retweets'])
User_id_dj = 24795163
followers_dj = 2136143
for each in actors_dict:
    if each['name'] == 'Dwayne Johnson':
        each['User_id'] = User_id_dj
        each['Posts'] = posts_dj
        each['Hashtags'] = hashtags_dj
        each['Posted_Time'] = posttime_dj
        each['Favourites'] = favs_dj
        each['Retweets'] = retweets_dj
        each['Followers'] = followers_dj

In [820]:
public_tweets_nosql = []        #Creating a list of dictionaries for public twitter data
for index,each in public_tweets_df.iterrows():
    public_tweets = dict()
    public_tweets['id'] = each['id']
    public_tweets['name'] = each['name']
    public_tweets['post'] = each['post']
    public_tweets['followers'] = each['followers']
    public_tweets_nosql.append(public_tweets)

### Converting facebook data to its NoSQL equivalent

In [829]:
prod_comp_fb_df = prod_comp_fb_df.reset_index(drop=True)

In [833]:
prodcomp_dict_list=[] #Creating a list of dictionaries for Production companies data scrapped from Facebook
prodcomp_dict=dict()
for index, row in prod_comp_fb_df.iterrows():  
    if row['Production_company'] in prodcomp_dict.keys():
        prodcomp_dict[row['Production_company']]['Posts'].append(row['Posts'])
        prodcomp_dict[row['Production_company']]['Hashtags'].append(row['Hashtags'])
        prodcomp_dict[row['Production_company']]['Likes'].append(row['Likes'])
        prodcomp_dict[row['Production_company']]['Shares'].append(row['Shares'])
        prodcomp_dict[row['Production_company']]['Comments'].append(row['Comments'])
        prodcomp_dict[row['Production_company']]['Created_time'].append(row['Created_time'])
    else:
        prodcomp_dict[row['Production_company']] = {'Page_likes':row['Page_likes'], 'Posts': [row['Posts']], \
                                                    'Hashtags':[row['Hashtags']] , 'Likes':[row['Likes']] ,\
                                                    'Shares':[row['Shares']], 'Comments':[row['Comments']], \
                                                    'Created_time':[row['Created_time']]
                                                   }

prod_comp_dict = []
for name in prodcomp_dict.keys():         #We then create a dictionary in the format as shown below with production companies
    d = prodcomp_dict[name]               #as the keys and its corresponding values
    d['Production_company'] = name
    prod_comp_dict.append(d)

In [839]:
public_facebook_nosql=[]
for index, row in facebook_public_data.iterrows():
    public_facebook=dict()
    public_facebook['name'] = row['name']
    public_facebook['followers/page_likes'] = row['followers/page_likes']
    public_facebook['post'] = row['post']
    public_facebook['post_likes'] = row['post_likes']
    public_facebook['shares'] = row['shares']
    public_facebook['comments'] = row['comments']
    public_facebook_nosql.append(public_facebook)
public_facebook_nosql

[{'comments': 255,
  'followers/page_likes': 50047182,
  'name': 'Disney',
  'post': '🚨 50 days. 🚨 #disney #Incredibles2 in theatres June 15. ',
  'post_likes': 2900,
  'shares': 732},
 {'comments': 0,
  'followers/page_likes': 2036311,
  'name': 'Helen Parr(Elastic Girl)',
  'post': 'See Elastigirl and The Incredibles in Disney Heroes : Battle Mode. Pre-register on Google Play today: http://per.blue/DisneyHeroesPreRegister See #Incredibles2 in theatres June 15 ',
  'post_likes': 40,
  'shares': 2},
 {'comments': 21,
  'followers/page_likes': 5173840,
  'name': 'IGN',
  'post': 'From Avengers: Infinity War to The Incredibles 2 to Slender Man and beyond, these are the must-see genre movies of Summer 2018!www.ign.com',
  'post_likes': 686,
  'shares': 27},
 {'comments': 0,
  'followers/page_likes': 2316,
  'name': 'Rtr Maitri Vasa',
  'post': '#Incredibles #November #disneystudios',
  'post_likes': 13,
  'shares': 0},
 {'comments': 0,
  'followers/page_likes': 769,
  'name': 'Anjaneya Sh

### Converting Instagram data to its NoSQL equivalent

In [854]:
genre_insta_df

,Genre,Tags,Counts
0,action,actionmovies,40224
1,action,actionmoviestar,4809
2,action,actionmoviestars,146
3,action,actionmoviestartv,64
4,action,actionmoviespictures,47
5,action,actionmoviesrock,40
6,action,actionmoviesarethebest,28
7,action,actionmovieshit,35
8,action,actionmovieschoolofenglish,42
9,action,actionmoviesfx,14


In [856]:
tags_dict_list=[]
tags_dict=dict()
for index, row in actors_insta_df.iterrows():  #First we create a dictionary with the actor name as key and his tags as values
    if row['Actor'] in tags_dict.keys():
        tags_dict[row['Actor']]['Tags'].append(row['Tags'])
        tags_dict[row['Actor']]['Count'].append(row['Counts'])
    else:
        tags_dict[row['Actor']] = {'Count':[row['Counts']], 'Tags': [row['Tags']]}

actors_dict = []
for name in tags_dict.keys():         #We then create a dictionary in the format as shown below with name,tags and count
    d = tags_dict[name]               # as the keys and its corresponding values
    d['Actor'] = name
    actors_dict.append(d)

In [859]:
genre_tags_dict_list=[]
tags_dict=dict()
for index, row in genre_insta_df.iterrows():  #First we create a dictionary with the Genre as key and his tags as values
    if row['Genre'] in tags_dict.keys():
        tags_dict[row['Genre']]['Tags'].append(row['Tags'])
        tags_dict[row['Genre']]['Count'].append(row['Counts'])
    else:
        tags_dict[row['Genre']] = {'Count':[row['Counts']], 'Tags': [row['Tags']]}

genre_dict = []
for name in tags_dict.keys():         #We then create a dictionary in the format as shown below with genre name
    d = tags_dict[name]               # as the keys and its corresponding values
    d['Genre'] = name
    genre_dict.append(d)

In [864]:
actors_dict

[{'Actor': 'Dwayne Johnson',
  'Count': [368759,
   4858,
   2007,
   672,
   795,
   535,
   537,
   466,
   718,
   452,
   443,
   413,
   405,
   632,
   416,
   407,
   473,
   451,
   448,
   406,
   408,
   412,
   408,
   464,
   191,
   148],
  'Tags': ['dwaynejohnson',
   'dwaynejohnsonfans',
   'dwaynejohnsonrocks',
   'dwaynejohnsonquotes',
   'dwaynejohnsontherock',
   'dwaynejohnson2020',
   'dwaynejohnsonfacts',
   'dwaynejohnsonmemes',
   'dwaynejohnsonofficial',
   'dwaynejohnsonstyle',
   'dwaynejohnsonmovies',
   'dwaynejohnsonpose',
   'dwaynejohnsontattoo',
   'dwaynejohnsonforpresident',
   'dwaynejohnsonday',
   'dwaynejohnsonart',
   'dwaynejohnsonwannabe',
   'dwaynejohnson_russia',
   'dwaynejohnsons',
   'dwaynejohnsonuniverse',
   'dwaynejohnsonbhramabull',
   'dwaynejohnsonworkout',
   'dwaynejohnsondrawing',
   'dwaynejohnsonismymotivation',
   'dwaynejohnsonfan',
   'dwaynejohnsonstar'],
  '_id': ObjectId('5ae3779afa65d8390c3a1cd6')},
 {'Actor': 'Brad Pit

## Establishing a MongoDB connection

Just as we import the sqlite library for sql part of this project, while using Jupyter notebook, same thing can be doe for the noSQL part as well by importig pymogo library. The sytax to write o SQL queries isn't much differet than sql.

In [73]:
import pymongo
from pymongo import MongoClient

client = MongoClient()
client.sampleDB

In [75]:
nosql_db = client.movies

In [321]:
nosql_db.actors.drop()
nosql_db.movies.drop()

In [323]:
nosql_db.movies.insert_many(movies_nosql_dict)
nosql_db.actors.insert_many(actors_dict)

In [841]:
nosql_db.public_tweets.insert_many(public_tweets_nosql)
nosql_db.production_companies_fb.insert_many(prod_comp_dict)
nosql_db.public_posts_fb.insert_many(public_facebook_nosql)

In [861]:
nosql_db.instagram_actors_tags.insert_many(actors_dict)
nosql_db.instagram_genres_tags.insert_many(genre_dict)

In [862]:
nosql_db.collection_names()

['production_companies_fb',
 'public_tweets',
 'movies',
 'public_posts_fb',
 'instagram_actors_tags',
 'instagram_genres_tags',
 'actors']

In [325]:
nosql_db.movies.count()

4774

# Project 2 - NoSQL 

## Answers Using Instagram Tags

## Q1 What are people saying about me ? (Johnny Depp)

We can answer this by searching for the tags related to Johnny Depp on Instagram

In [866]:
result2 = nosql_db.instagram_actors_tags.find_one({'Actor': 'Johnny Depp'})['Tags']
result2

['johnnydepp',
 'johnnydeppfans',
 'johnnydeppforever',
 'johnnydeppfan',
 'johnnydeppfanpage',
 'johnnydeppmovies',
 'johnnydepplookalike',
 'johnnydeppyoung',
 'johnnydepplove',
 'johnnydeppisinnocent',
 'johnnydepplovers',
 'johnnydeppquotes',
 'johnnydeppstyle',
 'johnnydeppneedsaoscar',
 'johnnydeppisperfect',
 'johnnydeppfandom',
 'johnnydeppedit',
 'johnnydeppishot',
 'johnnydeppislife',
 'johnnydeppily',
 'johnnydepppictures',
 'johnnydeppforlife',
 'johnnydepprp',
 'johnnydepphasourtrust',
 'johnnydeppmylove',
 'johnnydeppphotoshoot',
 'johnnydeppcosplay',
 'johnnydeppweloveyou',
 'johnnydepplover',
 'johnnydeppquote',
 'johnnydepptattoo',
 'johnnydepprare',
 'johnnydeppismygrindelwald',
 'johnnydeppedits',
 'johnnydeppart',
 'johnnydeppismylife',
 'johnnydeppsexy',
 'johnnydeppmovie',
 'johnnydeppdrawing',
 'johnnydepp_love_forever',
 'johnnydeppobsessed',
 'johnnydeppissexy',
 'johnnydeppfriends',
 'johnnydeppismine',
 'johnnydeppquetos',
 'johnnydeppdouble',
 'johnnydeppbr'

## Which actor is trending in your domain ?

In [376]:
result3 = nosql_db.actors.find_one({'Posted_Time': {'$regex':'2018'}, 'count':100})['name']
result3                                                              #We search for popular actors with posts in 2018

'Dwayne Johnson'

## Q3 How much influence do my (Anupam Kher) posts have ?

We can check Anupam Khers Followers and the number of favourites his posts get to understand his influence

In [363]:
result1 = np.mean(nosql_db.actors.find_one({'name':'Anupam Kher'})['Favourites'])
result2 = nosql_db.actors.find_one({'name':'Anupam Kher'})['Followers']   
print('Average Favourites per post = ',result1,',', 'Followers = ',result2)  

Average Favourites per post =  1885.77 , Followers =  12330588


<a rel="license" href="http://creativecommons.org/licenses/by/3.0/us/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/3.0/us/88x31.png" /></a><br />The text in the document by Hasher Hamza, Jhansi Nair, Parth Pandya is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/3.0/us/">Creative Commons Attribution 3.0 United States License</a>.

The code in the document by Hasher Hamza, Jhansi Nair, Parth Pandya is licensed under the MIT License https://opensource.org/licenses/MIT